
# **제6장. RAG and Agents**  

작업을 해결하기 위해서는 모델이 수행 방법에 대한 지침과 필요한 정보를 모두 알아야 합니다. 사람이 정보를 잃었을 때 잘못된 답변을 줄 가능성이 더 높듯이, AI 모델도 컨텍스트가 부족하면 실수를 하거나 환각(hallucinate)을 일으킬 가능성이 더 높습니다. 특정 애플리케이션의 경우, 모델의 지침은 모든 쿼리에 공통적이지만 컨텍스트는 각 쿼리에 따라 달라집니다. 지난 장에서는 모델에 대한 좋은 지침을 작성하는 방법에 대해 논의했습니다. 이번 장에서는 각 쿼리에 적합한 컨텍스트를 구성하는 방법에 초점을 맞춥니다.

컨텍스트를 구성하기 위한 두 가지 주요 패턴은 RAG(정보 검색 기반 생성)과 에이전트(agents)입니다. RAG 패턴은 모델이 외부 데이터 소스에서 관련 정보를 검색할 수 있도록 하며, 에이전트 패턴은 웹 검색 및 뉴스 API와 같은 도구를 사용하여 정보를 수집할 수 있게 합니다.

RAG 패턴이 주로 컨텍스트를 구성하는 데 사용되는 반면, 에이전트 패턴은 그 이상을 할 수 있습니다. 외부 도구는 모델이 자신의 단점을 보완하고 기능을 확장할 수 있도록 돕습니다. 가장 중요한 점은, 모델이 세계와 직접 상호작용할 수 있는 능력을 제공하여 우리의 삶의 많은 측면을 자동화할 수 있게 한다는 점입니다.

RAG와 에이전트 패턴은 이미 강력한 모델에 새로운 가능성을 제공하기 때문에 매우 흥미롭습니다. 짧은 시간 안에, 이들은 집단적 상상력을 사로잡아 놀라운 데모와 제품을 만들어내며 많은 사람들에게 미래를 보여주었습니다. 이번 장에서는 이러한 패턴 각각의 작동 방식, 그리고 이들이 주목받는 이유에 대해 자세히 다룰 것입니다.

--- 

## **RAG**  

RAG는 외부 메모리 소스에서 관련 정보를 검색하여 모델의 생성을 향상시키는 기술입니다. 외부 메모리 소스는 내부 데이터베이스, 사용자의 이전 채팅 세션 또는 인터넷일 수 있습니다.

"검색 후 생성(retrieve-then-generate)" 패턴은 “Reading Wikipedia to Answer Open-Domain Questions”(Chen et al., 2017)에서 처음 소개되었습니다. 이 작업에서는 시스템이 질문과 가장 관련이 있는 다섯 개의 Wikipedia 페이지를 검색한 다음, 모델이 이 페이지들의 정보를 사용하거나 읽어 답변을 생성하는 방식을 사용했습니다(그림 6-1 참조).

<img src="./images/fig_06_01.png" width="800">

‘정보 검색 기반 생성(Retrieval-Augmented Generation)’이라는 용어는 “Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks”(Lewis et al., 2020)에서 만들어졌습니다. 이 논문은 모든 가용 지식을 모델에 직접 넣을 수 없는 지식 집중형 작업에 대한 솔루션으로 RAG를 제안했습니다. RAG를 사용하면, 검색기가 결정한 쿼리와 가장 관련된 정보만 검색되고, 이것이 모델에 입력됩니다. Lewis 등은 관련 정보에 접근하는 것이 환각(hallucination)을 줄이면서 모델이 더 상세한 응답을 생성하는 데 도움을 줄 수 있다고 발견했습니다.

예를 들어, “Acme의 고급 프린터 A300이 100ppps로 인쇄할 수 있나요?”라는 질문에 대해, 모델은 프린터 A300의 사양을 제공받으면 더 나은 응답을 할 수 있습니다.

RAG를 각 쿼리에 특정한 컨텍스트를 구성하는 기술로 생각할 수 있습니다. 이는 동일한 컨텍스트를 모든 쿼리에 사용하는 대신, 사용자와 관련된 데이터만 포함하도록 사용자 데이터를 관리하는 데 도움을 줍니다.

기반 모델(foundation model)을 위한 컨텍스트 구성은 고전적인 머신러닝 모델의 특성 엔지니어링과 동일합니다. 둘 다 입력 데이터를 처리하는 데 필요한 정보를 모델에 제공하는 데 목적이 있습니다.

기반 모델 초기 단계에서 RAG는 가장 일반적인 패턴 중 하나로 등장했습니다. 그 주된 목적은 모델의 컨텍스트 한계를 극복하는 것이었습니다. 많은 사람들은 충분히 긴 컨텍스트 길이가 RAG의 끝이 될 것이라고 생각합니다. 하지만 저는 그렇게 생각하지 않습니다. 모델의 컨텍스트 길이가 아무리 길더라도, 그보다 긴 컨텍스트가 필요한 애플리케이션이 분명히 존재할 것입니다. 결국, 사용 가능한 데이터의 양은 시간이 지남에 따라 계속 증가합니다. 사람들은 데이터를 생성하고 새 데이터를 추가하지만 거의 삭제하지 않습니다. 컨텍스트 길이는 빠르게 확장되고 있지만, 임의의 애플리케이션이 필요로 하는 데이터 속도를 따라가지는 못하고 있습니다.

두 번째로, 긴 컨텍스트를 처리할 수 있는 모델이 반드시 그 컨텍스트를 잘 사용하는 것은 아닙니다. 이는 “컨텍스트 길이와 효율성(Context Length and Context Efficiency)”에서 논의된 바와 같이, 컨텍스트가 길어질수록 모델이 컨텍스트의 잘못된 부분에 집중할 가능성이 높아지기 때문입니다. 모든 추가 컨텍스트 토큰은 추가 비용을 초래하며, 잠재적으로 지연 시간을 증가시킬 수 있습니다. RAG는 각 쿼리에 대해 가장 관련성이 높은 정보를 사용하여 입력 토큰의 수를 줄이면서 모델의 성능을 잠재적으로 향상시킬 수 있습니다.

컨텍스트 길이를 확장하려는 노력은 모델이 컨텍스트를 더 효과적으로 사용하도록 만들기 위한 노력과 병행되고 있습니다. 모델이 검색 기반 또는 주의(attention-like) 메커니즘을 통합하여 쿼리에 대해 가장 관련성이 높은 컨텍스트를 선택하도록 하면 훨씬 효율적일 것입니다.

>**참고**
>
>Anthropic은 Claude 모델에 대해 "지식 기반의 크기가 20만 토큰(약 500페이지 분량의 자료)보다 작다면, 프롬프트에 전체 지식 기반을 포함시켜 모델에 제공하면 RAG 또는 유사한 방법이 필요 없을 수 있다"고 제안했습니다(Anthropic, 2024). 다른 모델 개발자들이 RAG와 긴 컨텍스트의 활용에 대한 비슷한 가이드를 제공한다면 정말 놀라울 것입니다.

---

### **RAG 아키텍처**

RAG 시스템은 두 가지 구성 요소로 이루어져 있습니다: 외부 메모리 소스에서 정보를 검색하는 검색기(retriever)와 검색된 정보를 기반으로 응답을 생성하는 생성기(generator)입니다. 그림 6-2는 RAG 시스템의 고급 아키텍처를 보여줍니다.

<img src="./images/fig_06_02.png" width="800">

초기 RAG 논문에서, Lewis et al.은 검색기와 생성기를 별도로 훈련했습니다. 오늘날의 RAG 시스템에서는 이 두 구성 요소가 종종 별도로 훈련되며, 많은 팀들이 기성 검색기와 모델을 사용하여 RAG 시스템을 구축합니다. 그러나 전체 RAG 시스템을 엔드투엔드(end-to-end) 방식으로 미세 조정하면 성능이 크게 향상될 수 있습니다.

RAG 시스템의 성공은 검색기의 품질에 크게 좌우됩니다. 검색기는 두 가지 주요 기능을 가지고 있습니다: **인덱싱**과 **쿼리 처리**입니다. 인덱싱은 데이터를 정리하여 나중에 빠르게 검색할 수 있도록 준비하는 과정입니다. 쿼리에 관련된 데이터를 보내는 과정을 **쿼리 처리**라고 합니다. 데이터를 어떻게 인덱싱할지는 나중에 데이터를 검색하려는 방식에 따라 다릅니다.

이제 주요 구성 요소를 다루었으니, RAG 시스템의 작동 예시를 살펴보겠습니다. 간단히 하기 위해, 외부 메모리 소스가 회사의 메모, 계약서, 회의 기록을 포함하는 데이터베이스라고 가정하겠습니다. 문서는 10개의 토큰일 수도 있고, 100만 개의 토큰일 수도 있습니다. 전체 문서를 단순히 검색하면 컨텍스트가 지나치게 길어질 수 있습니다. 이를 방지하기 위해, 각 문서를 더 관리하기 쉬운 청크로 나눌 수 있습니다. 청크로 나누는 전략은 이 장 후반에서 논의될 것입니다. 지금은 모든 문서가 작업 가능한 청크로 나뉘었다고 가정해봅시다. 각 쿼리에 대해, 우리의 목표는 이 쿼리와 가장 관련성이 높은 데이터 청크를 검색하는 것입니다. 사용자가 입력한 프롬프트와 검색된 데이터 청크를 결합해 최종 프롬프트를 생성하는 약간의 후처리가 필요할 수 있습니다. 이 최종 프롬프트는 이후 생성 모델에 입력됩니다.


>**참고**
>
>이 장에서는 "문서(document)"라는 용어를 "문서(document)"와 "청크(chunk)" 모두를 지칭하는 데 사용합니다. 기술적으로 문서의 청크도 문서로 간주될 수 있기 때문입니다. 이는 이 책의 용어가 고전적인 NLP 및 정보 검색(IR) 용어와 일치하도록 하기 위함입니다.

---


### **검색 알고리즘**

검색은 RAG에 국한된 것이 아닙니다. 정보 검색은 100년 이상의 역사를 가진 오래된 개념입니다. 이는 검색 엔진, 추천 시스템, 로그 분석 등 다양한 기술의 근간을 이룹니다. 전통적인 검색 시스템을 위해 개발된 많은 검색 알고리즘이 RAG에도 사용될 수 있습니다. 예를 들어, 정보 검색은 대규모 지원 산업을 가진 비옥한 연구 분야이므로 몇 페이지 안에 충분히 다룰 수 없습니다. 따라서 이 섹션에서는 개요만 다루며, 정보 검색에 대한 더 깊은 리소스는 이 책의 [GitHub 저장소](https://github.com/chiphuyen/aie-book)에서 확인할 수 있습니다.

>**참고**
>
>검색은 일반적으로 하나의 데이터베이스나 시스템에 국한되지만, 탐색(search)은 여러 시스템에 걸쳐 검색을 포함합니다. 이 장에서는 검색과 탐색을 혼용하여 사용합니다.


기본적으로, 정보 검색은 주어진 쿼리와의 관련성에 따라 문서를 순위 매기는 방식으로 작동합니다. 검색 알고리즘은 관련성을 계산하는 방식에 따라 달라집니다. 여기서는 두 가지 일반적인 검색 메커니즘인 **용어 기반 검색(term-based retrieval)** 과 **임베딩 기반 검색(embedding-based retrieval)** 부터 시작하겠습니다.


**희소(RAG) 대 밀집 검색(Sparse vs Dense Retrieval)**

문헌에서는 검색 알고리즘을 다음 두 가지 범주로 나누는 것을 볼 수 있습니다: 희소(sparse) 대 밀집(dense). 그러나 이 책은 용어 기반 검색(term-based retrieval)과 임베딩 기반 검색(embedding-based retrieval)이라는 범주화를 선택했습니다.

희소 검색기는 **희소 벡터(sparse vector)**를 사용하여 데이터를 표현합니다. 희소 벡터는 값의 대부분이 0인 벡터입니다. 용어 기반 검색은 각 용어가 **희소 원핫 벡터(one-hot vector)**로 표현될 수 있기 때문에 희소로 간주됩니다. 원핫 벡터는 대부분의 값이 0이고, 특정 위치에 1이 있는 벡터입니다. 벡터의 크기는 어휘(vocabulary)의 길이이며, 1의 값은 어휘에서 용어의 인덱스에 해당합니다.

예를 들어, 사전이 `{"food": 0, "banana": 1, "slug": 2}`와 같다면, "food", "banana", "slug"의 원핫 벡터는 각각 `[1, 0, 0]`, `[0, 1, 0]`, `[0, 0, 1]`이 됩니다.

밀집 검색기는 **밀집 벡터(dense vector)**를 사용하여 데이터를 표현합니다. 밀집 벡터는 값의 대부분이 0이 아닌 벡터입니다. 임베딩 기반 검색은 일반적으로 밀집으로 간주되며, 임베딩은 일반적으로 밀집 벡터입니다. 하지만, **희소 임베딩(sparse embedding)**도 존재합니다. 예를 들어, SPLADE(Sparse Lexical and Expansion)는 희소 임베딩을 사용하여 작동하는 검색 알고리즘입니다(Formal et al., 2021). SPLADE는 BERT에서 생성된 임베딩을 활용하며, 정규화를 통해 대부분의 임베딩 값을 0으로 밀어냅니다. 이 희소성은 임베딩 작업을 더 효율적으로 만듭니다.

희소 대 밀집 구분으로 인해 SPLADE는 용어 기반 알고리즘과 함께 분류됩니다. 그러나 SPLADE의 작업 방식, 강점 및 약점은 실제로는 밀집 임베딩 검색과 더 유사합니다. 용어 기반 대 임베딩 기반 구분은 이러한 잘못된 분류를 피할 수 있습니다.

**용어 기반 검색 (Term-based retrieval)**

쿼리가 주어졌을 때, 관련 문서를 찾는 가장 간단한 방법은 키워드를 사용하는 것입니다. 일부 사람들은 이를 **어휘 검색(lexical retrieval)**이라고 부릅니다. 예를 들어, “AI engineering”이라는 쿼리가 주어진 경우, 모델은 “AI engineering”을 포함하는 모든 문서를 검색합니다. 그러나 이 접근법에는 두 가지 문제가 있습니다:

- 많은 문서가 주어진 용어를 포함할 수 있으며, 모델이 이러한 모든 문서를 컨텍스트 공간에 포함하기에 충분하지 않을 수 있습니다. 일반적인 방법은 해당 용어를 가장 많이 포함하는 문서를 포함시키는 것입니다. 용어가 문서에 나타나는 횟수를 **용어 빈도(term frequency, TF)**라고 합니다.
  
- 프롬프트가 길고 많은 용어를 포함할 수 있습니다. 그러나 일부 용어는 다른 용어보다 더 중요합니다. 예를 들어, 프롬프트 “Easy-to-follow recipes for Vietnamese food to cook at home”는 다음과 같은 9개의 용어를 포함합니다: *easy-to-follow, recipes, for, vietnamese, food, to, cook, at, home.* 여기서 *vietnamese*와 *recipes*와 같은 더 중요한 용어에 집중하고 싶을 수 있습니다. 그러나 *for*와 *at*은 덜 중요한 용어로 간주됩니다. 중요한 용어를 식별할 방법이 필요합니다.

직관적으로, 더 많은 문서가 특정 용어를 포함할수록, 그 용어는 덜 유용하게 됩니다. 따라서 용어의 중요성은 그 용어가 등장하는 문서 수에 반비례합니다. 이 지표를 **역문서빈도(inverse document frequency, IDF)**라고 합니다. IDF를 계산하려면, 해당 용어를 포함하는 문서 수를 계산한 뒤, 총 문서 수를 이 값으로 나눕니다. 예를 들어, 10개의 문서가 있고, 특정 용어가 5개의 문서에 등장한다면, 이 용어의 IDF는 10/5 = 2가 됩니다. 용어의 IDF가 클수록 해당 용어는 더 중요합니다.

**TF-IDF**는 용어 빈도(TF)와 역문서빈도(IDF)라는 두 가지 지표를 결합하는 알고리즘입니다. 수학적으로, 쿼리 $ Q $에 대한 문서 $ D $의 TF-IDF 점수는 다음과 같이 계산됩니다:

1. $ t_1, t_2, ..., t_q $: 쿼리 $ Q $의 용어들.
2. 특정 용어 $ t $에 대해, 문서 $ D $에서의 용어 빈도 $ f(t, D) $.
3. $ N $: 전체 문서 수, $ C(t) $: 용어 $ t $를 포함하는 문서 수. $ t $의 IDF 값은 $ \text{IDF}(t) = \log N/C(t) $로 계산됩니다.
4. 단순히, 쿼리 $ Q $에 대한 문서 $ D $의 TF-IDF 점수는 $ \text{Score}(D, Q) = \sum_{i=1}^{q} \text{IDF}(t_i) \times f(t_i, D) $로 정의됩니다.

두 가지 일반적인 용어 기반 검색 솔루션은 **Elasticsearch**와 **BM25**입니다. **Elasticsearch**는 Lucene을 기반으로 만들어졌으며, 역인덱스(inverted index)를 사용합니다. 역인덱스는 용어에서 해당 용어를 포함하는 문서로 매핑하는 사전(dictionary)입니다. 이 인덱스는 특정 용어에 대한 문서를 빠르게 검색할 수 있도록 합니다. 이 사전은 용어 빈도, 용어가 등장하는 문서 수와 같은 추가 정보를 저장하여 TF-IDF 점수를 계산하는 데 유용합니다. **표 6-1**은 역인덱스를 보여줍니다.

**표 6-1. 역인덱스의 간단한 예시**

| 용어(Term) | 문서 개수(Document count) | (문서 인덱스, 용어 빈도) (Document index, term frequency) |
|------------|---------------------------|---------------------------------------------------------|
| banana     | 2                         | (10, 3), (5, 2)                                         |
| machine    | 4                         | (1, 5), (10, 1), (38, 9), (42, 5)                      |
| learning   | 3                         | (1, 5), (38, 7), (42, 5)                               |
| ...        | ...                       | ...                                                     |

Okapi BM25는 Best Matching 알고리즘의 25번째 버전으로, 1980년대 Robertson 등 연구자들에 의해 개발되었습니다. 이 스코어링 모델은 TF-IDF의 수정판입니다. 순수한 TF-IDF와 비교했을 때, BM25는 문서 길이에 따라 용어 빈도 점수를 정규화합니다. 더 긴 문서들은 특정 용어를 포함할 가능성이 더 높고, 더 높은 용어 빈도 값을 가질 수 있습니다.

BM25와 그 변형들(BM25+, BM25F)은 여전히 업계에서 널리 사용되고 있으며, 임베딩 기반 검색과 같은 현대적이고 정교한 검색 알고리즘과 비교하기 위한 강력한 기준점으로 작용합니다.

쿼리를 개별 용어로 분할하는 과정인 토크나이제이션(tokenization)에 대해 간략히 설명하자면, 가장 간단한 방법은 쿼리를 단어로 나누고 각 단어를 별개의 용어로 취급하는 것입니다. 그러나 이렇게 하면 다중 단어 용어가 개별 단어로 나뉘어 원래 의미를 잃을 수 있습니다. 예를 들어, "hot dog"은 "hot"과 "dog"으로 나뉘게 됩니다. 이렇게 되면 둘 중 어느 것도 원래 용어의 의미를 유지하지 못합니다. 이 문제를 완화하는 한 가지 방법은 가장 흔한 n-그램을 용어로 취급하는 것입니다. 예를 들어, "hot dog"이라는 이그램(bigram)이 일반적이라면, 이를 하나의 용어로 취급하게 됩니다.

추가적으로, 모든 문자를 소문자로 변환하거나, 구두점을 제거하거나, "the", "and", "is"와 같은 불용어(stop word)를 제거하는 것이 필요할 수도 있습니다. 용어 기반 검색 솔루션은 이러한 작업을 자동으로 처리합니다. NLTK(Natural Language Toolkit), spaCy, Stanford의 CoreNLP와 같은 고전적인 NLP 패키지도 토크나이제이션 기능을 제공합니다.

4장은 n-그램 중첩을 기준으로 두 텍스트 간의 어휘적 유사도를 측정하는 방법을 논의합니다. 쿼리와 문서가 서로 다른 길이를 가질 때, 쿼리의 n-그램을 포함하는 문서 수가 많아지면 많은 문서들이 비슷하게 높은 중첩 점수를 가지게 됩니다. 이로 인해 관련 없는 문서와 실제로 관련 있는 문서를 구별하기가 어려워질 수 있습니다.

**임베딩 기반 검색 (Embedding-based retrieval)**

용어 기반 검색은 텍스트의 의미보다는 어휘 수준에서 관련성을 계산합니다. 3장에서 언급했듯이, 텍스트의 외형적 특징은 반드시 의미를 포착하지 않습니다. 이는 의도와 무관한 문서를 반환하는 결과를 초래할 수 있습니다. 예를 들어, 쿼리 “transformer architecture”는 전기 기기나 영화 *Transformers*에 대한 문서를 반환할 수 있습니다. 반면에, **임베딩 기반 검색기(embedding-based retrievers)**는 쿼리와 얼마나 밀접하게 일치하는지에 따라 문서 순위를 매깁니다. 이러한 접근법은 **의미적 검색(semantic retrieval)**이라고도 합니다.

임베딩 기반 검색에서는 인덱싱에 추가적인 기능이 포함됩니다. 원래 데이터 청크를 임베딩으로 변환하는 것입니다. 생성된 임베딩이 저장되는 데이터베이스를 **벡터 데이터베이스(vector database)**라고 부릅니다. 쿼리는 다음 두 단계로 이루어집니다(그림 6-3 참조):

1. **임베딩 모델**: 인덱싱에 사용된 것과 동일한 임베딩 모델을 사용하여 쿼리를 임베딩으로 변환합니다.
2. **검색기**: 쿼리 임베딩과 가장 가까운 임베딩을 가진 데이터 청크를 검색합니다. 검색할 데이터 청크의 수 $ k $는 사용 사례, 생성 모델, 쿼리에 따라 달라집니다.

<img src="./images/fig_06_03.png" width="800">

위에서 보여준 임베딩 기반 검색 워크플로우는 간소화된 것입니다. 실제 의미적 검색 시스템은 모든 검색된 후보를 다시 정렬하는 **재정렬기(reranker)**, 캐시를 사용해 지연 시간을 줄이는 기능 등 다른 구성 요소를 포함할 수 있습니다.

임베딩 기반 검색을 사용하면, 3장에서 논의한 것처럼 다시 임베딩을 다루게 됩니다. 참고로, 임베딩은 원래 데이터의 중요한 속성을 보존하도록 설계된 벡터입니다. 임베딩 기반 검색기는 임베딩 모델이 좋지 않을 경우 이를 보완할 방법이 없습니다.

임베딩 기반 검색은 새로운 구성 요소인 **벡터 데이터베이스**를 도입합니다. 벡터 데이터베이스는 벡터를 저장합니다. 하지만 저장은 벡터 데이터베이스의 쉬운 부분입니다. 어려운 부분은 벡터 검색입니다. 주어진 쿼리 임베딩에 대해, 벡터 데이터베이스는 쿼리와 가까운 벡터를 찾아 반환해야 합니다. 벡터는 빠르고 효율적인 벡터 검색을 가능하게 하기 위해 인덱싱되고 저장되어야 합니다.

생성 AI 애플리케이션이 의존하는 다른 많은 메커니즘과 마찬가지로, 벡터 검색은 생성 AI에만 국한되지 않습니다. 벡터 검색은 검색, 추천, 데이터 조직화, 정보 검색, 클러스터링, 사기 탐지 등 임베딩을 사용하는 모든 애플리케이션에서 일반적으로 사용됩니다.

벡터 검색은 일반적으로 k-최근접 이웃 검색 문제(k-nearest neighbor search problem)로 공식화됩니다. 예를 들어, 주어진 쿼리에서 데이터베이스의 $ k $개의 가장 가까운 벡터를 찾는 것입니다. 기본 해결책은 다음과 같이 작동합니다:

1. 코사인 유사도와 같은 메트릭을 사용해 쿼리 임베딩과 데이터베이스의 모든 벡터 간의 유사도 점수를 계산합니다.
2. 유사도 점수에 따라 모든 벡터를 순위 매깁니다.
3. 가장 높은 유사도 점수를 가진 $ k $개의 벡터를 반환합니다.

이 단순한 해결책은 결과가 정확하도록 보장하지만 계산 비용이 많이 들고 느립니다. 따라서 작은 데이터셋에만 사용해야 합니다.

큰 데이터셋의 경우, 벡터 검색은 일반적으로 근사 최근접 이웃(ANN, Approximate Nearest Neighbor) 알고리즘을 사용하여 수행됩니다. 벡터 검색의 중요성 때문에, 많은 알고리즘과 라이브러리가 이를 위해 개발되었습니다. 인기 있는 벡터 검색 라이브러리로는 FAISS(Facebook AI Similarity Search, Johnson et al., 2017), Google의 ScaNN(Scalable Nearest Neighbors, Sun et al., 2020), Spotify의 Annoy(Bernhardsson, 2013), Hnswlib(Hierarchical Navigable Small World, Malkov and Yashunin, 2016) 등이 있습니다.

대부분의 애플리케이션 개발자는 직접 벡터 검색을 구현하지 않으므로, 다양한 접근 방식을 간략히 개요로 제공하겠습니다. 이 개요는 솔루션을 평가하는 데 유용할 수 있습니다.

일반적으로 벡터 데이터베이스는 벡터를 버킷(bucket), 트리(tree), 또는 그래프(graph)로 구성합니다. 벡터 검색 알고리즘은 유사한 벡터가 서로 가까이 위치하도록 가능성을 높이는 휴리스틱에 따라 다릅니다. 벡터는 또한 양자화(quantized)되어 정밀도를 줄이거나 희소하게 만들 수 있습니다. 양자화되고 희소한 벡터는 계산 비용이 더 적게 듭니다. 벡터 검색에 대해 더 자세히 알고 싶다면, Zilliz의 훌륭한 [시리즈](series 링크 참조)를 추천합니다. 아래는 주요 벡터 검색 알고리즘 몇 가지입니다:

- **LSH (Locality-Sensitive Hashing)** *(Indyk and Motwani, 1999)*  
    - 이 알고리즘은 단순한 벡터 이상을 다루는 강력하고 다재다능한 알고리즘입니다. 이는 유사한 벡터를 동일한 버킷으로 해싱하여 유사도 검색을 가속화하며, 정확성을 약간 희생하는 대신 효율성을 얻습니다. 이 알고리즘은 FAISS와 Annoy에 구현되어 있습니다.

- **HNSW (Hierarchical Navigable Small World)** *(Malkov and Yashunin, 2016)*  
    - HNSW는 노드가 벡터를 나타내고, 엣지가 유사한 벡터를 연결하는 다중 계층 그래프를 구성합니다. 이는 그래프 엣지를 탐색하여 최근접 이웃 검색을 수행합니다. 저자들이 제공한 구현은 오픈 소스이며, FAISS와 Milvus에도 구현되어 있습니다.

- **Product Quantization** *(Jégou et al., 2011)*  
    - 이 알고리즘은 각 벡터를 다수의 하위 벡터로 분해하여 훨씬 더 간단하고 저차원의 표현으로 줄이는 방식으로 작동합니다. 그런 다음, 저차원 표현을 사용해 거리를 계산하므로 작업 속도가 훨씬 빨라집니다. Product Quantization은 FAISS의 핵심 구성 요소이며, 거의 모든 인기 있는 벡터 검색 라이브러리에서 지원됩니다.

- **IVF (Inverted File Index)** *(Sivic and Zisserman, 2003)*  
    - IVF는 K-평균 클러스터링을 사용하여 유사한 벡터를 동일한 클러스터로 조직합니다. 데이터베이스의 벡터 수에 따라, 클러스터 수를 설정하여 평균적으로 각 클러스터에 100~10,000개의 벡터가 있도록 만듭니다. 쿼리 시, IVF는 쿼리 임베딩과 가장 가까운 클러스터 중심점을 찾고, 이 클러스터의 벡터를 후보 이웃으로 만듭니다. Product Quantization과 함께, IVF는 FAISS의 핵심 백본을 형성합니다.

- **Annoy (Approximate Nearest Neighbors Oh Yeah)** *(Bernhardsson, 2013)*  
    - Annoy는 다중 이진 트리를 구축합니다. 각 트리는 무작위로 선을 그어 벡터를 두 개의 분기로 분리하는 기준을 사용하여 벡터를 클러스터로 나눕니다. 검색 중에, 이 트리들을 탐색하여 후보 이웃을 선택합니다. Spotify는 이 구현을 오픈 소스로 제공하고 있습니다.

다른 알고리즘으로는 **Microsoft의 SPTAG** *(Space Partition Tree And Graph)*, **FLANN** *(Fast Library for Approximate Nearest Neighbors)* 등이 있습니다.

RAG의 대두와 함께 벡터 데이터베이스가 자체 카테고리로 자리 잡았지만, 데이터를 저장할 수 있는 모든 데이터베이스는 벡터 데이터베이스라고 불릴 수 있습니다. 많은 기존 데이터베이스가 벡터 저장 및 검색을 지원하도록 확장했거나 확장할 예정입니다.

**검색 알고리즘 비교**

검색의 긴 역사와 성숙한 솔루션 덕분에 용어 기반 검색(term-based retrieval)과 임베딩 기반 검색(embedding-based retrieval)은 비교적 쉽게 시작할 수 있습니다. 각각의 접근법에는 장단점이 있습니다.

용어 기반 검색은 인덱싱 및 쿼리 과정에서 임베딩 기반 검색보다 일반적으로 훨씬 빠릅니다. 용어 추출은 임베딩 생성보다 빠르고, 용어에서 해당 용어를 포함하는 문서로의 매핑은 최근접 이웃 검색보다 계산적으로 덜 비용이 많이 듭니다.

용어 기반 검색은 기본적으로 바로 사용할 수 있으며, Elasticsearch와 BM25 같은 솔루션은 많은 검색 및 검색 애플리케이션에서 성공적으로 사용되었습니다. 그러나 이 방법의 단순성은 조정 가능한 구성 요소가 적어 성능을 개선하기가 더 어렵다는 것을 의미하기도 합니다.

반면, 임베딩 기반 검색은 시간이 지나면서 성능이 개선되어 용어 기반 검색을 능가할 수 있습니다. 임베딩 모델과 검색기를 개별적으로, 함께, 또는 생성 모델과 결합하여 미세 조정할 수 있습니다. 그러나 데이터를 임베딩으로 변환하면 특정 오류 코드(예: EADDRNOTAVAIL (99))나 제품 이름과 같은 키워드를 흐리게 만들어 나중에 검색하기 어렵게 될 수 있습니다. 이러한 제한은 임베딩 기반 검색과 용어 기반 검색을 결합함으로써 해결할 수 있으며, 이는 이 장의 뒷부분에서 논의됩니다.

검색기의 품질은 검색된 데이터의 품질을 기준으로 평가할 수 있습니다. RAG 평가 프레임워크에서 자주 사용되는 두 가지 메트릭은 **컨텍스트 정밀도(context precision)**와 **컨텍스트 재현율(context recall)**입니다. (컨텍스트 정밀도는 **컨텍스트 관련성(context relevance)**이라고도 합니다):

- **컨텍스트 정밀도 (Context precision):**  
  검색된 모든 문서 중에서 쿼리와 관련 있는 문서의 비율은 얼마입니까?

- **컨텍스트 재현율 (Context recall):**  
  쿼리와 관련 있는 모든 문서 중에서 검색된 문서의 비율은 얼마입니까?

이 메트릭을 계산하기 위해, 테스트 쿼리 목록과 문서 집합이 포함된 평가 세트를 생성합니다. 각 쿼리에 대해, 각 문서가 관련 있는지 또는 관련 없는지를 주석으로 표시합니다. 이 주석 작업은 인간 또는 AI 판정자가 수행할 수 있습니다. 그런 다음 이 평가 세트를 사용하여 검색기의 정밀도와 재현율 점수를 계산합니다.

생산 환경에서 일부 RAG 프레임워크는 컨텍스트 정밀도만 지원하고 컨텍스트 재현율은 지원하지 않습니다. 특정 쿼리에 대한 재현율을 계산하려면 데이터베이스의 모든 문서가 해당 쿼리와 얼마나 관련이 있는지에 대한 주석이 필요합니다. 컨텍스트 정밀도는 더 간단하며, 검색된 문서가 쿼리와 얼마나 관련 있는지에만 주석을 달면 됩니다. 이 작업은 AI 판정자가 수행할 수 있습니다.

검색된 문서의 순위가 중요하다면, 예를 들어 더 관련 있는 문서가 상위에 있어야 한다면, NDCG(정규화된 누적 이익 점수), MAP(평균 정밀도), MRR(평균 역순위)와 같은 메트릭을 사용할 수 있습니다.

**의미적 검색**(semantic retrieval)을 위해서는 임베딩 품질도 평가해야 합니다. 3장에서 논의했듯이, 임베딩은 독립적으로 평가할 수 있으며, 유사한 문서가 더 가까운 임베딩 값을 가지면 좋은 것으로 간주됩니다. 임베딩은 특정 작업에 얼마나 잘 작동하는지에 따라서도 평가할 수 있습니다. MTEB 벤치마크(Muennighoff et al., 2023)는 검색, 분류, 클러스터링을 포함한 광범위한 작업에 대해 임베딩을 평가합니다.

검색기의 품질은 전체 RAG 시스템의 맥락에서 평가되어야 합니다. 궁극적으로, 검색기가 좋은지의 여부는 시스템이 고품질의 응답을 생성하는 데 얼마나 기여하는지에 따라 결정됩니다. 생성 모델의 출력 평가에 대한 내용은 3장과 4장에서 논의됩니다.

의미적 검색 시스템의 성능 이점이 비용 및 지연 시간 측면에서 얼마나 가치가 있는지는 쿼리 단계에서의 우선순위에 따라 다릅니다. RAG 지연 시간의 많은 부분은 출력 생성에서 발생하기 때문에, **쿼리 임베딩 생성 및 벡터 검색으로 인한 추가 지연 시간은 전체 RAG 지연 시간에 비해 최소일 수 있습니다.** 그럼에도 불구하고, 추가적인 지연 시간은 사용자 경험에 영향을 미칠 수 있습니다.

또 다른 문제는 비용입니다. 임베딩 생성은 비용이 듭니다. 특히 데이터가 자주 변경되며 임베딩을 다시 생성해야 하는 경우 문제가 될 수 있습니다. 매일 1억 개의 문서에 대해 임베딩을 생성해야 한다고 상상해 보십시오! 사용하는 벡터 데이터베이스에 따라 벡터 저장 및 벡터 검색 쿼리 역시 비용이 많이 들 수 있습니다. 일부 기업의 경우, 벡터 데이터베이스 비용이 모델 API 지출의 1/5 또는 절반에 달하기도 합니다.

**표 6-2**는 용어 기반 검색과 의미적 검색을 속도, 성능, 비용 측면에서 나란히 비교한 것입니다.

**표 6-2. 속도, 성능, 비용에 따른 용어 기반 검색과 의미적 검색 비교**

|                      | **용어 기반 검색 (Term-based retrieval)**          | **임베딩 기반 검색 (Embedding-based retrieval)**          |
|----------------------|--------------------------------------------------|-------------------------------------------------------|
| **쿼리 속도**         | 임베딩 기반 검색보다 훨씬 빠름                      | 쿼리 임베딩 생성과 벡터 검색은 느릴 수 있음                  |
| **성능**             | 기본적으로 강력한 성능, 그러나 개선하기 어려움          | 미세 조정으로 용어 기반 검색을 능가할 수 있음<br>용어 대신 의미에 중점을 둬 자연어 쿼리 사용 가능 |
| **비용**             | 임베딩 기반 검색보다 훨씬 저렴함                     | 임베딩 생성, 벡터 저장, 벡터 검색 솔루션은 비용이 많이 들 수 있음          |


검색 시스템에서는 인덱싱과 쿼리 처리 간에 특정 트레이드오프를 설정할 수 있습니다. 인덱스가 더 세부적일수록 검색 프로세스는 더 정확해지지만, 인덱싱 프로세스는 더 느리고 메모리를 더 많이 소모합니다. 예를 들어, 잠재 고객 목록을 작성한다고 상상해 보십시오. 더 많은 세부 정보를 추가하면(예: 이름, 회사, 이메일, 전화번호, 관심사 등), 관련 있는 사람을 더 쉽게 찾을 수 있지만, 이를 생성하는 데 시간이 더 오래 걸리고 더 많은 저장 공간이 필요합니다.

일반적으로, HNSW(Hierarchical Navigable Small World)와 같은 세부적인 인덱스는 높은 정확성과 빠른 쿼리 시간을 제공하지만, 이를 구축하는 데 상당한 시간과 메모리가 필요합니다. 반면, LSH(Locality-Sensitive Hashing)와 같은 더 간단한 인덱스는 생성이 더 빠르고 메모리 소모가 적지만, 쿼리는 더 느리고 정확성이 낮아집니다.

**[ANN-Benchmarks 웹사이트](ANN-Benchmarks 웹사이트 링크)**는 여러 데이터셋에서 다양한 ANN 알고리즘을 비교하며, 인덱싱과 쿼리 처리 간의 트레이드오프를 고려하는 네 가지 주요 메트릭을 사용합니다. 이 메트릭은 다음과 같습니다:

- **Recall (재현율):**  
  알고리즘이 찾은 최근접 이웃의 비율.

- **Query per second (QPS):**  
  알고리즘이 초당 처리할 수 있는 쿼리의 수. 이는 트래픽이 많은 애플리케이션에서 매우 중요합니다.

- **Build time (빌드 시간):**  
  인덱스를 구축하는 데 필요한 시간. 데이터가 자주 변경되는 경우, 이 메트릭은 특히 중요합니다.

- **Index size (인덱스 크기):**  
  알고리즘이 생성한 인덱스의 크기. 이는 확장성과 저장 요구 사항을 평가하는 데 중요합니다.

추가적으로, **BEIR (Benchmarking IR)** *(Thakur et al., 2021)*은 검색 평가를 위한 하네스(harness)로, 14개의 일반적인 검색 벤치마크에서 검색 시스템을 지원합니다.

요약하면, RAG 시스템의 품질은 구성 요소별로, 그리고 시스템 전반에서 평가되어야 합니다. 이를 위해 다음 작업을 수행해야 합니다:

1. 검색 품질을 평가하십시오.  
2. 최종 RAG 출력을 평가하십시오.  
3. 임베딩 기반 검색의 경우 임베딩을 평가하십시오.  

**검색 알고리즘 결합**

서로 다른 검색 알고리즘의 뚜렷한 장점을 고려할 때, 실제 검색 시스템은 여러 접근 방식을 결합하는 경우가 많습니다. 용어 기반 검색과 임베딩 기반 검색을 결합하는 것을 **하이브리드 검색(hybrid search)**이라고 합니다.

서로 다른 알고리즘은 순차적으로 사용할 수 있습니다. 먼저, 용어 기반 시스템과 같은 저렴하고 덜 정확한 검색기를 사용하여 후보를 가져옵니다. 그런 다음, k-최근접 이웃과 같은 더 정밀하지만 비용이 많이 드는 메커니즘을 사용하여 이러한 후보 중에서 가장 적합한 것을 찾습니다. 이 두 번째 단계는 **재정렬(reranking)**이라고도 합니다.

예를 들어, “transformer”라는 용어가 주어지면, 전기 기기, 신경망 구조, 또는 영화를 포함하여 “transformer”라는 단어를 포함하는 모든 문서를 가져올 수 있습니다. 그런 다음 벡터 검색을 사용하여 이 문서들 중에서 실제로 트랜스포머 쿼리와 관련이 있는 문서를 찾습니다. 또 다른 예로, “X에서 가장 많은 매출을 책임지고 있는 사람은 누구인가?”라는 쿼리를 고려해 보십시오. 먼저, 키워드 X를 사용하여 X와 관련된 모든 문서를 가져온 다음, 벡터 검색을 사용하여 “가장 많은 매출을 책임지고 있는 사람”이라는 컨텍스트를 검색합니다.

서로 다른 알고리즘은 앙상블로 병렬로도 사용할 수 있습니다. 검색기는 쿼리에 대한 관련성 점수에 따라 문서를 순위 매기므로, 여러 검색기를 동시에 사용하여 후보를 가져온 다음, 이러한 다른 순위를 결합하여 최종 순위를 생성할 수 있습니다.

서로 다른 순위를 결합하는 알고리즘은 **역순위 융합(Reciprocal Rank Fusion, RRF)** *(Cormack et al., 2009)*이라고 합니다. 이 알고리즘은 각 문서에 검색기의 순위에 따라 점수를 할당합니다. 직관적으로, 문서가 1위에 랭크되면 점수는 $1/1 = 1$이 되고, 2위에 랭크되면 점수는 $1/2 = 0.5$가 됩니다. 순위가 높을수록 점수가 높아집니다.

문서의 최종 점수는 모든 검색기에 대해 계산된 점수의 합입니다. 예를 들어, 한 검색기에서 1위로 랭크되고 다른 검색기에서 2위로 랭크된 문서의 점수는 $1 + 0.5 = 1.5$가 됩니다. 이는 RRF의 기본을 보여주는 간단한 예이며, 실제 공식은 다음과 같이 더 복잡합니다:

$
\text{Score}(D) = \sum_{i=1}^n \frac{1}{k + r_i(D)}
$

여기서:  
- $n$: 순위 목록의 수; 각 순위 목록은 검색기에 의해 생성됩니다.  
- $r_i(D)$: 검색기 $i$에 의해 문서 $D$가 매겨진 순위.  
- $k$: 0으로 나누는 것을 방지하고 하위 순위 문서의 영향을 제어하기 위한 상수. 일반적인 $k$ 값은 60입니다.

---

### **검색 최적화 (Retrieval Optimization)**

작업에 따라 특정 전술이 관련 문서를 검색할 가능성을 높일 수 있습니다. 여기서 다루는 네 가지 전술은 청크 전략(chunking strategy), 재정렬(reranking), 쿼리 재작성(query rewriting), 컨텍스트 검색(contextual retrieval)입니다.


**청크 전략 (Chunking strategy)**

데이터를 나누는 방법은 나중에 데이터를 검색하려는 방식에 따라 달라집니다. 이전 섹션에서는 서로 다른 검색 알고리즘과 관련된 인덱싱 전략을 다루었습니다. 거기에서는 문서가 이미 관리 가능한 청크로 나뉘어 있다고 가정했습니다. 이 섹션에서는 다양한 청크 전략에 대해 다룰 것입니다. 이는 사용하는 청크 전략이 검색 시스템의 성능에 크게 영향을 미칠 수 있기 때문에 중요한 고려사항입니다.

가장 간단한 전략은 특정 단위를 기준으로 동일한 길이의 청크로 문서를 나누는 것입니다. 일반적인 단위는 문자, 단어, 문장, 단락입니다. 예를 들어, 각 문서를 2,048자나 512단어 청크로 나눌 수 있습니다. 각 청크가 일정한 수의 문장(예: 20문장) 또는 단락(예: 단락당 하나의 청크)을 포함하도록 설정할 수도 있습니다.

또한, 점점 더 작은 단위로 문서를 재귀적으로 나눌 수도 있습니다. 예를 들어, 문서를 섹션으로 나누고, 섹션이 너무 길면 단락으로 나누며, 단락이 여전히 너무 길면 문장으로 나누는 방식입니다. 이는 관련 텍스트가 임의로 잘리는 것을 방지합니다.

특정 문서는 더 창의적인 청크 전략이 필요할 수 있습니다. 예를 들어, 질문-답변(Q&A) 문서는 각 질문 또는 답변을 하나의 청크로 나눌 수 있습니다. 중국어 텍스트는 영어 텍스트와 다르게 나눌 필요가 있을 수 있습니다.

문서를 겹치지 않게 청크로 나누면 중요한 텍스트가 중간에서 잘릴 수 있어 중요한 정보 손실로 이어질 수 있습니다. 예를 들어, 텍스트 "I left my wife a note"를 "I left my wife"와 "a note"로 나누면, 두 청크 중 어느 것도 원래 텍스트의 핵심 정보를 완전히 전달하지 못합니다. 겹치기를 사용하면 중요한 경계 정보가 최소한 하나의 청크에 포함되도록 보장할 수 있습니다. 청크 크기를 2,048자로 설정한 경우, 겹치기 크기를 20자로 설정할 수 있습니다.

청크 크기는 생성 모델의 최대 컨텍스트 길이를 초과해서는 안 됩니다. 임베딩 기반 접근법에서는 청크 크기가 임베딩 모델의 컨텍스트 제한을 초과해서도 안 됩니다.

또한, 청크를 생성 모델의 토크나이저에 의해 결정되는 토큰 단위로 나눌 수 있습니다. 예를 들어, Llama 3를 생성 모델로 사용하려는 경우, Llama 3의 토크나이저를 사용하여 먼저 텍스트를 토큰화한 다음, 이 토큰을 경계로 삼아 청크를 나눌 수 있습니다. 토큰 단위로 청크를 나누면 다운스트림 모델과 작업하기 더 쉬워집니다. 그러나 다른 토크나이저를 사용하는 생성 모델로 전환하면 데이터를 다시 인덱싱해야 할 수 있다는 단점이 있습니다.

어떤 전략을 선택하든, 청크 크기는 중요합니다. 작은 청크 크기는 더 다양한 정보를 포함할 수 있습니다. 작은 청크 크기는 모델의 컨텍스트에 더 많은 청크를 맞출 수 있습니다. 청크 크기를 절반으로 줄이면 두 배의 청크를 맞출 수 있으며, 이는 모델이 더 넓은 범위의 정보를 활용하여 더 나은 답변을 제공할 수 있도록 합니다.

그러나 작은 청크 크기는 중요한 정보 손실을 초래할 수 있습니다. 예를 들어, 문서 전체에서 X에 대한 중요한 정보를 포함하고 있지만, X가 처음 절반에만 언급된 경우, 문서를 두 청크로 나누면 두 번째 절반은 검색되지 않아 모델이 해당 정보를 사용할 수 없게 될 수 있습니다.

작은 청크 크기는 계산적 오버헤드도 증가시킬 수 있습니다. 특히 임베딩 기반 검색에서는 더욱 그렇습니다. 청크 크기를 반으로 줄이면 인덱싱 및 검색해야 할 청크 수가 두 배가 되고, 벡터 검색 공간도 두 배가 되어 쿼리 속도를 줄일 수 있습니다.

최적의 청크 크기 또는 겹치기 크기에 대한 보편적인 규칙은 없습니다. 최적의 설정을 찾으려면 실험이 필요합니다.

**재정렬 (Reranking)**

검색기가 생성한 초기 문서 순위는 더 정확하게 재정렬될 수 있습니다. 재정렬은 검색된 문서 수를 줄여 모델의 컨텍스트에 맞추거나 입력 토큰 수를 줄여야 할 때 특히 유용합니다.

재정렬에 대한 일반적인 패턴은 **"검색 알고리즘 결합"** 섹션에서 논의됩니다. 저렴하지만 덜 정밀한 검색기가 후보를 가져온 다음, 더 정밀하지만 비용이 많이 드는 메커니즘이 이 후보들을 재정렬합니다.

문서는 시간에 따라 재정렬될 수도 있으며, 더 최근 데이터를 더 높은 가중치로 부여합니다. 이는 뉴스 집계, 이메일과의 채팅(예: 이메일에 대한 질문에 답변할 수 있는 챗봇), 또는 주식 시장 분석과 같은 시간에 민감한 애플리케이션에 유용합니다.

컨텍스트 재정렬은 기존 검색 재정렬과는 다릅니다. 검색에서는 항목의 정확한 위치(예: 1위 또는 5위)가 중요합니다. 반면, 컨텍스트 재정렬에서는 문서 순서가 여전히 중요하지만, 이는 모델이 문서를 얼마나 잘 처리할 수 있는지에 영향을 미치기 때문입니다. 모델은 컨텍스트의 시작과 끝에 있는 문서를 더 잘 이해할 수 있습니다. 이는 **"컨텍스트 길이와 컨텍스트 효율성"**에서 논의된 바 있습니다. 그러나 문서가 포함되기만 하면, 검색 순서의 영향은 검색 순위와 비교해 덜 중요합니다.

**쿼리 재작성 (Query rewriting)**

쿼리 재작성은 쿼리 재구성(query reformulation), 쿼리 정규화(query normalization), 때로는 쿼리 확장(query expansion)이라고도 불립니다. 다음 대화를 고려해 보십시오:

- **사용자:** John Doe가 마지막으로 우리에게서 뭔가를 산 게 언제인가요?  
- **AI:** John은 2030년 1월 3일, 2주 전에 Fruity Fedora 모자를 마지막으로 우리에게서 샀습니다.  
- **사용자:** Emily Doe는요?

마지막 질문, "Emily Doe는요?"는 컨텍스트 없이 애매합니다. 이 쿼리를 그대로 사용하여 문서를 검색하면 관련 없는 결과를 얻을 가능성이 높습니다. 사용자가 실제로 묻고자 하는 내용을 반영하도록 이 쿼리를 재작성해야 합니다. 새로운 쿼리는 자체적으로 의미를 가져야 합니다. 이 경우, 쿼리는 "Emily Doe가 마지막으로 우리에게서 뭔가를 산 게 언제인가요?"로 재작성되어야 합니다.

쿼리 재작성을 **RAG**의 범주로 포함했지만, 쿼리 재작성은 RAG에 국한되지 않습니다. 전통적인 검색 엔진에서도 쿼리 재작성은 자주 휴리스틱을 사용하여 이루어집니다. AI 애플리케이션에서는 "다음 대화를 바탕으로, 사용자가 실제로 묻고자 하는 내용을 반영하도록 마지막 사용자 입력을 재작성하십시오."와 같은 프롬프트를 사용하여 다른 AI 모델로도 쿼리 재작성을 수행할 수 있습니다. **그림 6-4**는 ChatGPT가 이 프롬프트를 사용하여 쿼리를 재작성한 방법을 보여줍니다.

<img src="./images/fig_06_04.png" width="800">

쿼리 재작성은 특히 신원 확인(identity resolution)을 수행하거나 다른 지식을 통합해야 할 경우 복잡해질 수 있습니다. 예를 들어, 사용자가 "그의 아내는요?"라고 묻는다면, 먼저 데이터베이스를 쿼리하여 그의 아내가 누구인지 알아내야 합니다. 만약 이 정보를 가지고 있지 않다면, 재작성 모델은 잘못된 이름을 만들어내어(환각하여) 틀린 답변을 초래하기보다는 이 쿼리가 해결할 수 없는 것임을 인정해야 합니다.

**컨텍스트 검색 (Contextual retrieval)**

컨텍스트 검색의 아이디어는 각 청크에 관련된 컨텍스트를 추가하여 관련 청크를 더 쉽게 검색할 수 있도록 하는 것입니다. 간단한 기술은 태그나 키워드와 같은 메타데이터를 사용하여 청크를 보강하는 것입니다. 전자상거래의 경우, 제품은 설명 및 리뷰로 보강될 수 있습니다. 이미지와 비디오는 제목이나 캡션으로 쿼리할 수 있습니다.

메타데이터에는 청크에서 자동으로 추출된 엔터티도 포함될 수 있습니다. 문서에 특정 용어(예: 오류 코드 EADDRNOTAVAIL (99))가 포함된 경우, 문서의 메타데이터에 이를 추가하면 문서가 임베딩으로 변환된 후에도 시스템이 이 키워드를 통해 이를 검색할 수 있습니다.

또한, 각 청크가 답변할 수 있는 질문을 추가할 수 있습니다. 예를 들어, 고객 지원의 경우, 비밀번호 재설정 방법에 대한 기사는 "비밀번호를 어떻게 재설정하나요?", "비밀번호를 잊어버렸어요", "로그인할 수 없어요", 또는 "도와주세요, 내 계정을 찾을 수 없어요"와 같은 쿼리를 추가하여 보강할 수 있습니다.

문서를 여러 청크로 나눈 경우, 일부 청크는 검색기가 청크의 내용을 이해하는 데 필요한 컨텍스트가 부족할 수 있습니다. 이를 방지하려면 원래 문서의 제목 및 요약과 같은 컨텍스트를 각 청크에 추가할 수 있습니다. **Anthropic**은 AI 모델을 사용하여 50~100 토큰 정도의 짧은 컨텍스트를 생성했습니다. 이 컨텍스트는 청크와 원래 문서의 관계를 설명합니다. Anthropic이 이를 위해 사용한 프롬프트는 다음과 같습니다:

```
<document>
{{WHOLE_DOCUMENT}}
</document>

Here is the chunk we want to situate within the whole document:

<chunk>
{{CHUNK_CONTENT}}
</chunk>

Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk. Answer only with the succinct context and nothing else.
```

생성된 컨텍스트는 각 청크에 추가되고, 보강된 청크는 검색 알고리즘에 의해 인덱싱됩니다. **그림 6-5**는 Anthropic이 따르는 프로세스를 시각화한 것입니다.

<img src="./images/fig_06_05.png" width="800">

**검색 솔루션 평가 (Evaluating Retrieval Solutions)**

검색 솔루션을 평가할 때 염두에 두어야 할 몇 가지 주요 요소는 다음과 같습니다:

- 어떤 검색 메커니즘을 지원하나요? 하이브리드 검색을 지원하나요?  
- 벡터 데이터베이스인 경우, 어떤 임베딩 모델과 벡터 검색 알고리즘을 지원하나요?  
- 데이터 저장 및 쿼리 트래픽 측면에서 얼마나 확장 가능합니까? 귀하의 트래픽 패턴에 적합한가요?  
- 데이터를 인덱싱하는 데 얼마나 시간이 걸리나요? 한 번에 대량의 데이터를 처리할 수 있나요(예: 추가/삭제)?  
- 다양한 검색 알고리즘에 대한 쿼리 지연 시간은 얼마인가요?  
- 관리형 솔루션인 경우, 가격 구조는 어떻게 되나요? 문서/벡터 볼륨 기준인가요, 아니면 쿼리 볼륨 기준인가요?

이 목록에는 액세스 제어, 준수, 데이터 플레인 및 컨트롤 플레인 분리와 같은 엔터프라이즈 솔루션과 일반적으로 연관된 기능은 포함되어 있지 않습니다.

**검색 솔루션 평가 (Evaluating Retrieval Solutions)**

검색 솔루션을 평가할 때 염두에 두어야 할 몇 가지 주요 요소는 다음과 같습니다:

- 어떤 검색 메커니즘을 지원하나요? 하이브리드 검색을 지원하나요?  
- 벡터 데이터베이스인 경우, 어떤 임베딩 모델과 벡터 검색 알고리즘을 지원하나요?  
- 데이터 저장 및 쿼리 트래픽 측면에서 얼마나 확장 가능합니까? 귀하의 트래픽 패턴에 적합한가요?  
- 데이터를 인덱싱하는 데 얼마나 시간이 걸리나요? 한 번에 대량의 데이터를 처리할 수 있나요(예: 추가/삭제)?  
- 다양한 검색 알고리즘에 대한 쿼리 지연 시간은 얼마인가요?  
- 관리형 솔루션인 경우, 가격 구조는 어떻게 되나요? 문서/벡터 볼륨 기준인가요, 아니면 쿼리 볼륨 기준인가요?

이 목록에는 액세스 제어, 준수, 데이터 플레인 및 컨트롤 플레인 분리와 같은 엔터프라이즈 솔루션과 일반적으로 연관된 기능은 포함되어 있지 않습니다.

---


### **텍스트를 넘어선 RAG (RAG Beyond Texts)**

이전 섹션에서는 외부 데이터 소스로 텍스트 문서를 사용하는 텍스트 기반 RAG 시스템에 대해 논의했습니다. 그러나 외부 데이터 소스는 멀티모달(multi-modal) 데이터와 표 형식(tabular) 데이터일 수도 있습니다.

**멀티모달 RAG (Multimodal RAG)**

생성기가 멀티모달인 경우, 컨텍스트는 텍스트 문서뿐만 아니라 이미지, 비디오, 오디오 등 외부 소스를 통해 보강될 수 있습니다. 글을 간결하게 유지하기 위해 예시로 이미지를 사용하겠지만, 이미지를 다른 형식의 모달리티로 교체할 수 있습니다. 쿼리가 주어지면 검색기는 해당 쿼리와 관련된 텍스트와 이미지를 가져옵니다. 예를 들어, "픽사 영화 *업*에 나오는 집의 색깔은 무엇인가요?"라는 질문이 주어진 경우, 검색기는 *업*에 나오는 집의 사진을 가져와 모델의 답변을 돕습니다. 이는 **그림 6-6**에 나와 있습니다.

<img src="./images/fig_06_06.png" width="800">

이미지에 제목, 태그, 캡션과 같은 메타데이터가 있는 경우, 메타데이터를 사용하여 검색할 수 있습니다. 예를 들어, 이미지의 캡션이 쿼리와 관련이 있다고 판단되면 해당 이미지를 검색할 수 있습니다.

이미지의 내용을 기준으로 이미지를 검색하려면 쿼리와 이미지를 비교할 수 있는 방법이 필요합니다. 쿼리가 텍스트인 경우, 텍스트와 이미지를 모두 임베딩할 수 있는 멀티모달 임베딩 모델이 필요합니다. 예를 들어, CLIP *(Radford et al., 2021)*을 멀티모달 임베딩 모델로 사용한다고 가정합니다. 검색기는 다음과 같이 작동합니다:

1. 모든 데이터(텍스트와 이미지)에 대해 CLIP 임베딩을 생성하고, 이를 벡터 데이터베이스에 저장합니다.
2. 쿼리가 주어지면, 해당 쿼리의 CLIP 임베딩을 생성합니다.
3. 벡터 데이터베이스에서 쿼리 임베딩과 가까운 텍스트와 이미지의 임베딩을 검색합니다.

**표 형식 데이터와 함께하는 RAG (RAG with Tabular Data)**

대부분의 애플리케이션은 텍스트와 이미지 같은 비구조적 데이터뿐만 아니라 표 형식 데이터도 다룹니다. 많은 쿼리가 데이터베이스의 정보가 필요할 수 있습니다. 표 형식 데이터를 사용하여 컨텍스트를 보강하는 워크플로우는 기존 RAG 워크플로우와 크게 다릅니다.

고양이 패션을 전문으로 하는 전자상거래 사이트 **Kitty Vogue**에서 근무한다고 가정해봅시다. 이 상점은 아래 **표 6-3**과 같은 주문 테이블 **Sales**를 가지고 있습니다.

**표 6-3. 가상의 전자상거래 사이트 Kitty Vogue의 주문 테이블 예시**

| 주문 ID(Order ID) | 타임스탬프(Timestamp) | 제품 ID(Product ID) | 제품(Product)      | 단가(Unit price $) | 수량(Units) | 총액(Total) |
|-------------------|-----------------------|---------------------|--------------------|--------------------|-------------|-------------|
| 1                 | 2044                  | Meow Mix Seasoning  | 10.99             | 1                 | 10.99        |
| 2                 | 3492                  | Purr & Shake         | 25                | 2                 | 50           |
| 3                 | 2045                  | Fruity Fedora        | 18                | 1                 | 18           |
| ...               | ...                   | ...                 | ...               | ...               | ...          |

사용자가 “지난 7일 동안 Fruity Fedora는 몇 개나 팔렸나요?”라는 질문을 했다고 가정하면, 시스템은 이 테이블을 쿼리하여 Fruity Fedora를 포함한 모든 주문을 찾고, 모든 주문의 수량을 합산해야 합니다. 이 테이블이 SQL로 쿼리될 수 있다고 가정하면, SQL 쿼리는 다음과 같이 보일 수 있습니다:

```sql
SELECT SUM(units) AS total_units_sold
FROM Sales
WHERE product_name = 'Fruity Fedora'
AND timestamp >= DATE_SUB(CURDATE(), INTERVAL 7 DAY);
```

워크플로우는 다음과 같으며, 이는 **그림 6-7**에 시각화되어 있습니다. 이 워크플로우를 실행하려면, 시스템이 SQL 쿼리를 생성하고 실행할 수 있는 기능을 가져야 합니다:

1. **텍스트에서 SQL로 (Text-to-SQL):** 사용자 쿼리와 제공된 테이블 스키마를 기반으로 필요한 SQL 쿼리를 결정합니다. 텍스트에서 SQL로의 변환은 **2장**에서 논의된 의미적 파싱(semantic parsing)의 예입니다.
2. **SQL 실행 (SQL execution):** SQL 쿼리를 실행합니다.
3. **생성 (Generation):** SQL 결과와 원래 사용자 쿼리를 기반으로 응답을 생성합니다.

<img src="./images/fig_06_07.png" width="800">

모델 컨텍스트에 모든 테이블 스키마를 포함시킬 수 없는 많은 테이블이 있을 경우, 각 쿼리에 대해 어떤 테이블을 사용할지 예측하는 중간 단계가 필요할 수 있습니다. Text-to-SQL은 동일한 생성기 또는 특화된 Text-to-SQL 모델을 사용하여 수행할 수 있습니다.

이 섹션에서는 검색기 및 SQL 실행기와 같은 도구가 모델이 더 많은 쿼리를 처리하고 더 높은 품질의 응답을 생성할 수 있도록 어떻게 활성화하는지 논의했습니다. 모델에 더 많은 도구에 대한 액세스를 제공하는 것이 그 기능을 더욱 향상시킬 수 있을까요? 도구 사용은 **에이전트 패턴(agentic pattern)**의 핵심 특성 중 하나이며, 이는 다음 섹션에서 논의할 것입니다.

---

## **에이전트 (Agents)**

지능형 에이전트는 많은 사람들이 AI의 궁극적인 목표로 간주합니다. Stuart Russell과 Peter Norvig의 고전적인 저서 *Artificial Intelligence: A Modern Approach* (Prentice Hall, 1995)는 합리적인 에이전트를 연구하고 설계하는 분야로 인공지능 연구를 정의합니다.

기반 모델의 전례 없는 능력은 이전에는 상상할 수 없었던 에이전트 애플리케이션의 문을 열었습니다. 이러한 새로운 능력은 자율적이고 지능적인 에이전트를 개발하여 우리의 비서, 동료, 코치 역할을 하게 하는 것을 가능하게 합니다. 이들은 웹사이트를 만들고, 데이터를 수집하고, 여행을 계획하고, 시장 조사를 수행하며, 고객 계정을 관리하고, 데이터 입력을 자동화하고, 인터뷰 준비를 돕고, 후보자를 인터뷰하며, 거래를 협상하는 등 다양한 일을 할 수 있습니다. 가능성은 무궁무진하며, 이러한 에이전트의 잠재적인 경제적 가치는 막대합니다.

>**경고 (WARNING)**
>
>AI 기반 에이전트는 정의, 개발 및 평가를 위한 이론적 프레임워크가 아직 정립되지 않은 신흥 분야입니다. 이 섹션은 기존 문헌을 기반으로 프레임워크를 구축하려는 최선의 시도입니다. 그러나 이 분야가 발전함에 따라 계속 발전할 것입니다. 책의 다른 부분과 비교했을 때, 이 섹션은 더 실험적인 성격을 띱니다.

이 섹션은 에이전트 개요로 시작한 다음, 에이전트의 능력을 결정짓는 두 가지 측면(도구와 계획)으로 이어질 것입니다. 새로운 작동 방식을 갖춘 에이전트는 새로운 실패 방식도 가지고 있습니다. 이 섹션은 이러한 실패를 포착하기 위해 에이전트를 평가하는 방법에 대한 논의로 끝날 것입니다.

에이전트는 새로운 개념이지만, 이 책에서 이미 언급된 자기 비판(self-critique), 사고의 연결(chain-of-thought), 구조화된 출력과 같은 개념에 기반을 두고 있습니다.

---

### **에이전트 개요 (Agent Overview)**

`에이전트(agent)`라는 용어는 소프트웨어 에이전트, 지능형 에이전트, 사용자 에이전트, 대화형 에이전트, 강화 학습 에이전트를 포함하여 여러 공학적 맥락에서 사용되었습니다. 그렇다면 에이전트란 무엇일까요?

에이전트는 환경을 인식하고 그 환경에서 행동할 수 있는 모든 것입니다. 이는 에이전트가 활동하는 **환경(environment)**과 수행할 수 있는 **행동 집합(set of actions)**에 의해 특징지어진다는 것을 의미합니다.

- **환경(environment):**  
  에이전트가 활동하는 환경은 그 용도에 따라 정의됩니다. 예를 들어, 게임을 하도록 개발된 에이전트라면(예: *Minecraft*, *Go*, *Dota*), 그 게임이 에이전트의 환경입니다. 인터넷에서 문서를 스크랩하는 에이전트라면 인터넷이 환경입니다. 요리 로봇인 에이전트의 환경은 주방이며, 자율 주행차 에이전트의 환경은 도로 시스템과 그 인접 지역입니다.

- **행동 집합(set of actions):**  
  AI 에이전트가 수행할 수 있는 행동은 사용할 수 있는 **도구(tools)**에 의해 확장됩니다. 많은 생성형 AI 기반 애플리케이션은 간단한 도구이긴 하지만 도구에 접근할 수 있는 에이전트입니다. 예를 들어, ChatGPT는 에이전트로 인터넷 검색, Python 코드 실행, 이미지 생성 등의 작업을 수행할 수 있습니다. RAG 시스템 또한 에이전트이며, 텍스트 검색기, 이미지 검색기, SQL 실행기 등이 이들의 도구입니다.

에이전트의 환경과 행동 집합은 강한 의존 관계를 가지고 있습니다. 환경은 에이전트가 수행할 수 있는 유효한 행동을 결정합니다. 예를 들어, 환경이 체스 게임이라면 에이전트의 유효한 행동은 체스 규칙에 따라 움직이는 것입니다. 반대로, 에이전트의 도구 목록은 그 환경을 제한할 수 있습니다. 예를 들어, 로봇의 유일한 행동이 수영이라면, 그 로봇은 물 환경으로만 제한됩니다.

**그림 6-8**은 GPT-4를 기반으로 한 SWE 에이전트 *(Yang et al., 2024)*의 시각화를 보여줍니다. 이 에이전트의 환경은 터미널과 파일 시스템이 있는 컴퓨터입니다. 이 에이전트의 행동 집합은 리포지토리 탐색, 파일 검색, 파일 보기, 라인 편집을 포함합니다.

<img src="./images/fig_06_08.png" width="800">

AI 에이전트는 보통 사용자가 입력한 작업을 완료하기 위해 설계됩니다. AI 에이전트에서 "뇌" 역할은 입력 정보(작업 및 환경으로부터의 피드백 포함)를 처리하고, 작업을 달성하기 위한 행동의 순서를 계획하며, 작업이 완료되었는지 판단합니다.

**예시: RAG 시스템과 표 형식 데이터**  
Kitty Vogue의 예시로 돌아가 보겠습니다. 이 에이전트는 세 가지 행동(응답 생성, SQL 쿼리 생성, SQL 쿼리 실행)을 가진 단순한 에이전트입니다. "Fruity Fedora의 향후 3개월 판매 수익을 예측하세요."라는 쿼리가 주어졌을 때, 에이전트는 다음과 같은 일련의 행동을 수행할 수 있습니다:

1. 이 작업을 어떻게 수행할지 추론합니다. 향후 판매를 예측하려면 최근 5년간의 판매 데이터를 먼저 가져와야 한다고 판단할 수 있습니다. 에이전트의 추론은 중간 응답으로 표시됩니다.
2. 최근 5년간의 판매 데이터를 가져오기 위한 쿼리를 생성하기 위해 SQL 쿼리 생성을 호출합니다.
3. 이 쿼리를 실행하기 위해 SQL 쿼리 실행을 호출합니다.
4. 결과 출력에 대해 추론하고, 이 결과가 판매 예측에 어떻게 도움이 되는지 판단합니다. 이 숫자가 불충분하다고 판단할 수 있습니다. 예를 들어, 누락된 값 때문에 정확한 예측을 하기에 충분하지 않다고 판단할 수 있습니다. 따라서 과거 마케팅 캠페인에 대한 정보도 필요하다고 결정합니다.
5. 과거 마케팅 캠페인에 대한 쿼리를 생성하기 위해 SQL 쿼리 생성을 호출합니다.
6. 이 쿼리를 실행하기 위해 SQL 쿼리 실행을 호출합니다.
7. 새로운 정보가 향후 판매를 예측하기에 충분하다고 추론합니다. 이후 예측을 생성합니다.
8. 작업이 성공적으로 완료되었음을 판단합니다.

에이전트가 아닌 사용 사례와 비교했을 때, 에이전트는 보통 더 강력한 모델이 필요합니다. 그 이유는 다음 두 가지입니다:

- **누적되는 실수(Compound mistakes):** 에이전트는 작업을 완료하기 위해 여러 단계를 수행해야 하는 경우가 많으며, 단계 수가 증가할수록 전체 정확도가 감소합니다. 모델의 단일 단계 정확도가 95%라면, 10단계를 거치면 정확도는 60%로 떨어지며, 100단계를 거치면 정확도는 0.6%에 불과합니다.

- **더 높은 중요도(Higher stakes):** 도구에 접근할 수 있는 에이전트는 더 영향력 있는 작업을 수행할 수 있지만, 실패할 경우 더 심각한 결과를 초래할 수 있습니다.

여러 단계를 요구하는 작업은 실행하는 데 시간과 비용이 많이 들 수 있습니다. 그러나 에이전트가 자율적으로 작동할 수 있다면, 많은 인간의 시간을 절약하여 그 비용을 가치 있게 만들 수 있습니다.

주어진 환경에서 에이전트의 성공 여부는 에이전트가 접근할 수 있는 도구 인벤토리와 AI 계획기의 강점에 달려 있습니다. 모델이 사용할 수 있는 다양한 종류의 도구를 살펴보는 것으로 시작해 봅시다.

---

**도구 (Tools)**

시스템은 에이전트가 되기 위해 외부 도구에 접근할 필요는 없습니다. 그러나 외부 도구가 없다면, 에이전트의 능력은 제한될 것입니다. 모델 자체로는 일반적으로 한 가지 작업만 수행할 수 있습니다. 예를 들어, 대규모 언어 모델(LLM)은 텍스트를 생성할 수 있고, 이미지 생성기는 이미지를 생성할 수 있습니다. 외부 도구는 에이전트를 훨씬 더 강력하게 만듭니다.

도구는 에이전트가 환경을 인식하고 행동할 수 있도록 돕습니다. 환경을 인식하는 행동은 읽기 전용(read-only) 작업인 반면, 환경에서 행동할 수 있는 작업은 쓰기 작업(write actions)입니다.

이 섹션에서는 외부 도구에 대한 개요를 제공합니다. 도구를 사용하는 방법은 **‘계획(Planning)’**에서 논의됩니다.

에이전트가 접근할 수 있는 도구 집합은 도구 인벤토리(tool inventory)입니다. 에이전트의 도구 인벤토리는 에이전트가 무엇을 할 수 있는지를 결정하므로, 에이전트에게 어떤 도구를 얼마나 제공할지 신중하게 생각하는 것이 중요합니다. 더 많은 도구가 더 많은 기능을 에이전트에게 제공하지만, 도구가 많을수록 이를 잘 이해하고 활용하는 것이 더 어려워집니다. **‘도구 선택’**에서 논의된 것처럼, 적절한 도구 집합을 찾기 위한 실험이 필요합니다.

에이전트의 환경에 따라 가능한 도구는 다양합니다. 여기에는 고려해야 할 세 가지 도구 범주가 포함될 수 있습니다: 지식 보강(knowledge augmentation, 즉 컨텍스트 구축), 기능 확장(capability extension), 그리고 에이전트가 환경에서 행동할 수 있도록 하는 도구입니다.

---

**지식 보강 (Knowledge augmentation)**

이 책이 지금까지 모델 응답 품질을 위해 적절한 컨텍스트를 갖는 것이 얼마나 중요한지 설득했다고 생각합니다. 도구의 중요한 범주 중 하나는 에이전트의 지식을 보강하는 데 도움을 주는 도구입니다. 텍스트 검색기, 이미지 검색기, SQL 실행기와 같은 도구가 이미 논의되었습니다. 추가적인 잠재적 도구로는 내부 인물 검색, 다양한 제품 상태를 반환하는 재고 API, Slack 리트리버, 이메일 리더 등이 포함됩니다.

이러한 많은 도구는 모델을 조직의 비공개 프로세스 및 정보와 결합합니다. 그러나 도구는 인터넷과 같은 공공 정보에 대한 접근도 제공합니다.

웹 브라우징은 ChatGPT와 같은 챗봇에 통합될 수 있는 가장 초기의 기대되는 기능 중 하나였습니다. 웹 브라우징은 모델이 오래된 상태(stale)가 되는 것을 방지합니다. 모델이 훈련된 데이터가 최신 상태가 아니면, 모델은 예를 들어 지난주에 끊긴 데이터와 관련된 질문에 답할 수 없습니다. 웹 브라우징이 없다면 모델은 날씨, 뉴스, 예정된 이벤트, 주식 가격, 비행 상태 등에 대해 알려줄 수 없습니다.

저는 웹 브라우징을 인터넷에 접근하는 모든 도구를 포괄하는 용어로 사용합니다. 여기에는 웹 브라우저와 검색 API, 뉴스 API, GitHub API, X(구 Twitter), LinkedIn, Reddit과 같은 소셜 미디어 API가 포함됩니다.

웹 브라우징은 에이전트가 더 나은 응답을 생성하고 환각(hallucination)을 줄이기 위해 최신 정보를 참조할 수 있게 합니다. 그러나 이는 또한 인터넷의 부정확한 정보에 에이전트를 노출시킬 수 있습니다. 인터넷 API를 신중하게 선택하십시오.

---

**기능 확장 (Capability extension)**

고려해야 할 두 번째 도구 범주는 AI 모델의 고유한 한계를 해결하는 데 초점을 맞춥니다. 이러한 도구는 모델의 성능을 향상시키는 쉬운 방법을 제공합니다. 예를 들어, AI 모델은 수학에 약한 것으로 악명이 높습니다. 모델에 "199,999를 292로 나눈 값은 무엇인가요?"라고 묻는다면, 모델은 실패할 가능성이 큽니다. 그러나 모델이 계산기에 접근할 수 있다면 이 계산은 간단해집니다. 모델을 산술에 능숙하게 훈련시키는 대신, 계산기 도구를 제공하는 것이 훨씬 더 자원 효율적입니다.

모델의 기능을 크게 향상시킬 수 있는 간단한 도구로는 달력, 시간대 변환기, 단위 변환기(예: 파운드를 킬로그램으로 변환) 및 모델이 잘 다루지 못하는 언어 간 번역이 포함됩니다.

더 복잡하지만 강력한 도구는 코드 해석기입니다. 모델이 코드를 이해하도록 훈련시키는 대신, 코드 해석기에 접근 권한을 제공하여 코드를 실행하고, 결과를 반환하며, 코드의 오류를 분석할 수 있도록 할 수 있습니다. 이러한 기능은 에이전트를 코딩 비서, 데이터 분석가, 심지어 연구 비서로 작동하게 만듭니다. 연구 비서는 실험을 위한 코드를 작성하고 결과를 보고할 수 있습니다. 그러나 자동화된 코드 실행은 코드 삽입 공격의 위험을 수반하므로, **‘Defensive Prompt Engineering’**에서 논의된 대로 적절한 보안 조치가 필수적입니다.

외부 도구는 텍스트 전용 또는 이미지 전용 모델을 멀티모달로 만들 수 있습니다. 예를 들어, 텍스트만 생성할 수 있는 모델은 텍스트-이미지 생성 모델을 도구로 활용하여 텍스트와 이미지를 모두 생성할 수 있습니다. 텍스트 요청이 주어지면, 에이전트의 AI 계획자는 텍스트 생성, 이미지 생성 또는 둘 다 수행할지 결정합니다. ChatGPT가 텍스트와 이미지를 모두 생성할 수 있는 방식이 바로 이것입니다. ChatGPT는 DALL-E를 이미지 생성기로 사용합니다. 에이전트는 차트와 그래프를 생성하기 위해 코드 해석기를 사용할 수도 있고, 수학 방정식을 렌더링하기 위해 LaTeX 컴파일러를, 또는 HTML 코드에서 웹 페이지를 렌더링하기 위해 브라우저를 사용할 수도 있습니다.

마찬가지로, 텍스트 입력만 처리할 수 있는 모델은 이미지 캡션 도구를 사용하여 이미지를 처리하거나, 트랜스크립션 도구를 사용하여 오디오를 처리할 수 있습니다. OCR(광학 문자 인식) 도구를 사용하여 PDF를 읽을 수도 있습니다.

**도구 사용은 단순한 프롬프트 전달이나 정교한 미세 조정(finetuning)과 비교했을 때 모델의 성능을 크게 향상시킬 수 있습니다.** Chameleon *(Lu et al., 2023)* 연구는 13개의 도구 세트를 사용해 보강된 GPT-4 기반 에이전트가 여러 벤치마크에서 GPT-4 단독으로 성능을 능가할 수 있음을 보여줍니다. 이 에이전트가 사용하는 도구의 예로는 지식 검색, 쿼리 생성기, 이미지 캡션 도구, 텍스트 감지기, 검색 엔진 등이 포함됩니다.

ScienceQA(과학 질문 응답 벤치마크)에서 Chameleon은 발표된 최고 수준의 few-shot 결과를 11.37% 향상시켰습니다. TabMWP(표 형식 수학 문제 벤치마크)에서 Chameleon은 정확도를 17% 향상시켰습니다.

---

**쓰기 작업 (Write actions)**

지금까지 데이터 소스를 읽을 수 있는 읽기 전용 작업에 대해 논의했습니다. 그러나 도구는 데이터를 읽는 것뿐만 아니라 쓰기 작업도 수행할 수 있습니다. 예를 들어, SQL 실행기는 데이터 테이블을 읽거나, 테이블에 행을 추가하거나 삭제하는 작업(쓰기)을 수행할 수 있습니다. 이메일 API는 이메일을 읽을 수 있을 뿐만 아니라 이메일에 응답할 수도 있습니다. 은행 API는 현재 잔액을 검색할 수 있을 뿐만 아니라 은행 송금을 시작할 수도 있습니다.

쓰기 작업은 시스템이 더 많은 일을 할 수 있도록 합니다. 예를 들어, 잠재 고객을 찾고, 그들의 연락처 정보를 확보하고, 이메일 초안을 작성하고, 첫 번째 이메일을 보내고, 응답을 추적하며, 계약을 체결하고, 데이터베이스를 새 주문으로 업데이트하는 전체 고객 아웃리치 워크플로를 자동화할 수 있습니다.

그러나 AI가 자동으로 우리의 삶을 변경하도록 허용하는 것은 두려운 전망일 수 있습니다. 생산 데이터베이스에 대한 권한을 신뢰할 수 없는 AI에 제공하지 않는 것처럼, 신뢰할 수 없는 AI가 은행 송금을 시작하도록 허용해서는 안 됩니다. 시스템의 기능을 신뢰하고, 시스템이 유해한 작업을 수행하지 않도록 보호하기 위해 보안 조치를 취하는 것이 중요합니다.

자율 AI 에이전트에 대해 이야기할 때, 종종 자율주행차와 관련된 질문이 나옵니다. 예를 들어, "누군가 해킹해서 차를 납치하면 어떻게 될까요?" 자율주행차는 물리적 특성 때문에 실제 세계에서 더 생생하게 느껴질 수 있지만, 물리적 세계에 존재하지 않아도 AI 시스템은 주식 시장을 조작하거나, 저작권을 침해하거나, 사생활을 침해하거나, 편향을 강화하거나, 허위 정보를 퍼뜨릴 수 있습니다. 이에 대해서는 **‘Defensive Prompt Engineering’**에서 논의된 바 있습니다.

이 모든 우려는 타당하며, AI를 활용하려는 조직은 안전과 보안을 진지하게 고려해야 합니다. 그러나 이것이 AI 시스템이 실제 세계에서 행동할 능력을 결코 얻지 말아야 한다는 것을 의미하지는 않습니다. 언젠가 사람들이 AI를 믿고 우주로 데려갈 수 있다면, 보안 조치가 자율 AI 시스템을 신뢰할 수 있을 만큼 충분해질 것이라고 희망합니다. 물론, 인간도 실패할 수 있습니다. 개인적으로 저는 낯선 사람이 차를 운전하는 것보다는 자율주행차를 더 믿을 것입니다.

적절한 도구는 사람들이 훨씬 더 생산적이 되도록 도울 수 있습니다. 예를 들어, 엑셀 없이 사업을 하거나, 크레인 없이 고층 건물을 짓는 것을 상상할 수 있습니까? 도구는 모델이 훨씬 더 많은 작업을 수행할 수 있도록 합니다. 많은 모델 제공업체는 이미 도구 사용을 지원하고 있으며, 이를 **함수 호출(function calling)**이라고 부릅니다. 앞으로는 함수 호출을 지원하는 도구 세트가 대부분의 모델에서 일반적으로 사용될 것으로 예상됩니다.

---

### **계획 (Planning)**

기반 모델 에이전트의 핵심은 작업을 해결하는 데 책임이 있는 모델입니다. 작업은 목표와 제약 조건으로 정의됩니다. 예를 들어, 작업은 샌프란시스코에서 인도로 가는 2주간의 여행을 계획하고, 예산은 5,000달러인 것입니다. 목표는 2주간의 여행이고, 제약 조건은 예산입니다.

복잡한 작업은 계획이 필요합니다. 계획 프로세스의 출력은 작업을 완료하기 위해 필요한 단계를 설명하는 로드맵인 **계획(plan)**입니다. 효과적인 계획은 모델이 작업을 이해하고, 이를 달성하기 위한 다양한 옵션을 고려하며, 가장 유망한 옵션을 선택하도록 요구합니다.

만약 당신이 어떤 계획 회의에라도 참석한 적이 있다면, 계획이 어렵다는 것을 알 것입니다. 중요한 계산 문제로서, 계획은 철저히 연구되어야 하며, 이를 다루기 위해 여러 권의 책이 필요할 것입니다. 여기서는 표면만 간략히 다룰 수 있습니다.

---

**계획 개요 (Planning Overview)**

주어진 작업을 처리하는 데는 여러 가지 방법이 있지만, 그 중 모든 방법이 성공적인 결과를 보장하는 것은 아닙니다. 올바른 해결책 중에는 더 효율적인 방법도 있습니다. 예를 들어, "수익이 없지만 10억 달러 이상을 모금한 기업은 몇 개입니까?"라는 질문을 고려해 봅시다. 이를 해결하는 여러 방법이 있지만, 예시로 두 가지 옵션을 고려합니다:

1. 수익이 없는 모든 회사를 찾은 후, 그중에서 모금 금액으로 필터링합니다.
2. 10억 달러 이상을 모금한 모든 회사를 찾은 후, 그중에서 수익으로 필터링합니다.

두 번째 옵션이 더 효율적입니다. 수익이 없는 회사는 10억 달러 이상을 모금한 회사보다 훨씬 더 많기 때문입니다. 이 두 가지 옵션만 주어진 경우, 지능적인 에이전트는 두 번째 옵션을 선택해야 합니다.

계획과 실행을 동일한 프롬프트에서 결합할 수도 있습니다. 예를 들어, 모델에 프롬프트를 제공하고, 사고의 연결(chain-of-thought) 프롬프트와 같이 한 단계씩 생각하도록 요청한 다음, 해당 단계를 모두 한 번에 실행하도록 요청할 수 있습니다. 그러나 모델이 목표를 달성하지 못하는 1,000단계짜리 계획을 세운다면 어떻게 될까요? 감독 없이 에이전트는 몇 시간 동안 API 호출에 시간과 비용을 낭비하다가, 아무 소득 없이 중단될 수 있습니다.

이러한 무익한 실행을 방지하려면, **계획(planning)**은 실행과 분리되어야 합니다. 에이전트에게 먼저 계획을 생성하도록 요청하고, 이 계획이 검증(validated)된 후에만 실행하도록 해야 합니다. 계획은 휴리스틱(heuristics)을 사용하여 검증될 수 있습니다. 예를 들어, 하나의 간단한 휴리스틱은 잘못된 행동을 포함하는 계획을 제거하는 것입니다. 생성된 계획이 Google 검색을 필요로 하지만 에이전트가 Google 검색에 접근할 수 없는 경우, 이 계획은 유효하지 않습니다. 또 다른 휴리스틱은 X단계를 초과하는 모든 계획을 제거하는 것입니다. AI 심판(AI judges)을 사용하여 계획을 검증할 수도 있습니다. 모델에 계획이 합리적인지, 또는 개선 방법을 평가하도록 요청할 수 있습니다.

생성된 계획이 나쁘다고 평가되면, 계획자(planner)에게 다른 계획을 생성하도록 요청할 수 있습니다. 생성된 계획이 좋다고 평가되면, 실행합니다. 계획에 외부 도구가 포함되어 있다면, 함수 호출(function calling)이 필요합니다. 이 계획에서 얻어진 출력은 다시 평가되어야 합니다. 생성된 계획은 전체 작업에 대한 끝에서 끝까지(end-to-end) 계획일 필요는 없습니다. 하위 작업에 대한 작은 계획일 수도 있습니다. 전체 프로세스는 **그림 6-9**와 같습니다.

<img src="./images/fig_06_09.png" width="800">

시스템에는 이제 세 가지 구성 요소가 있습니다: 계획을 생성하는 구성 요소, 계획을 검증하는 구성 요소, 그리고 계획을 실행하는 구성 요소. 각 구성 요소를 에이전트로 간주하면, 이것은 다중 에이전트 시스템(multi-agent system)입니다. 

프로세스를 가속화하려면, 계획을 순차적으로 생성하는 대신, 여러 계획을 병렬로 생성하고 평가자(evaluator)가 가장 유망한 것을 선택하도록 요청할 수 있습니다. 이는 또 다른 대기 시간/비용 절충(latency/cost trade-off)이며, 여러 계획을 동시에 생성하는 데 추가 비용이 발생할 수 있습니다.

**계획에는 작업 뒤에 숨겨진 의도를 이해하는 것이 필요합니다:**  사용자가 이 질문으로 무엇을 하려고 하는가? 의도 분류기는 종종 에이전트가 계획을 세우는 데 도움을 주기 위해 사용됩니다. **‘Break Complex Tasks into Simpler Subtasks’**에서 설명된 것처럼, 의도 분류는 이 작업을 위해 다른 프롬프트를 사용하거나 특별히 훈련된 분류 모델을 통해 수행할 수 있습니다. 의도 분류 메커니즘은 다중 에이전트 시스템의 또 다른 에이전트로 간주될 수 있습니다.

의도를 아는 것은 에이전트가 올바른 도구를 선택하는 데 도움을 줄 수 있습니다. 예를 들어, 고객 지원의 경우, 질문이 결제와 관련되어 있다면 에이전트는 사용자의 최근 결제를 검색하기 위해 도구에 접근해야 할 수 있습니다. 하지만 질문이 비밀번호 재설정 방법에 관한 것이라면, 에이전트는 문서 검색 도구가 필요할 수 있습니다.

---

>**TIP:**  
>
>일부 쿼리는 에이전트의 범위를 벗어날 수 있습니다. 의도 분류기는 요청을 "관련 없음(IRRELEVANT)"으로 분류할 수 있어야 하며, 에이전트가 불가능한 해결책을 생각하느라 자원을 낭비하는 대신 정중히 요청을 거절할 수 있어야 합니다.

---

지금까지 우리는 에이전트가 모든 세 단계를 자동화한다고 가정했습니다: 계획 생성, 계획 검증, 계획 실행. 그러나 실제로는, 인간이 이 프로세스에서 위험을 줄이고 돕기 위해 어느 단계에서든 참여할 수 있습니다. 예를 들어, 인간 전문가가 계획을 제공하거나, 계획을 검증하거나, 계획의 일부를 실행할 수 있습니다. 에이전트가 전체 계획을 생성하는 데 어려움을 겪는 복잡한 작업의 경우, 인간 전문가는 에이전트가 동의할 수 있는 고수준의 계획을 제공할 수 있습니다. 계획이 데이터베이스 업데이트나 코드 변경 병합과 같은 위험한 작업을 포함한다면, 시스템은 실행 전에 명시적인 인간 승인을 요청하거나, 인간이 이러한 작업을 수행하도록 허용할 수 있습니다. 이를 가능하게 하려면, 각 작업에 대해 에이전트가 가질 수 있는 자동화 수준을 명확히 정의해야 합니다.

**요약하자면, 작업을 해결하는 일반적인 과정은 다음과 같습니다:**  
반영(reflection)은 에이전트에게 필수적인 것은 아니지만, 에이전트의 성능을 크게 향상시킬 것입니다.

1. **계획 생성 (Plan generation):**  
   이 작업을 완료하기 위한 계획을 수립합니다. 계획은 관리 가능한 작업의 순서입니다. 이 과정은 작업 분해(task decomposition)라고도 합니다.

2. **반영 및 오류 수정 (Reflection and error correction):**  
   생성된 계획을 평가합니다. 계획이 나쁘다면, 새로운 계획을 생성합니다.

3. **실행 (Execution):**  
   생성된 계획에 명시된 작업을 실행합니다. 여기에는 특정 함수 호출이 포함될 수 있습니다.

4. **반영 및 오류 수정 (Reflection and error correction):**  
   작업 결과를 검토한 후, 결과를 평가하고 목표가 달성되었는지 확인합니다. 목표가 완료되지 않은 경우, 문제를 식별하고 수정하며 새 계획을 생성합니다.

이 책에서 계획 생성과 반영에 대한 몇 가지 기술을 이미 살펴보셨을 겁니다. 모델에게 "단계별로 생각하라"고 요청할 때, 이는 작업을 분해하도록 요청하는 것입니다. 모델에게 "당신의 답변이 정확한지 확인하십시오"라고 요청하는 것은 반영을 요청하는 것입니다.

---

**플래너로서의 기반 모델 (Foundation models as planners)**

기반 모델이 얼마나 잘 계획할 수 있는지는 여전히 풀리지 않은 질문입니다. 많은 연구자들은 적어도 오토레그레시브 언어 모델을 기반으로 한 모델은 계획할 수 없다고 믿습니다. Meta의 수석 AI 과학자인 Yann LeCun은 2023년 **"오토레그레시브 LLM은 계획할 수 없다"**고 명백히 언급했습니다. **"Can LLMs Really Reason and Plan?"**이라는 기사에서 Kambhampati(2023)는 LLM이 지식을 추출하는 데는 훌륭하지만, 계획하는 데는 부족하다고 주장합니다. 그는 논문에서 계획과 관련된 지식을 실행 가능한 계획으로 변환하는 것이 LLM의 문제라고 지적합니다. **"LLM에서 나오는 계획은 사용자에게는 그럴듯하게 보이지만, 실행 시간 상호작용과 오류로 이어질 수 있다."**

그러나 LLM이 형편없는 플래너라는 일화적인 증거가 많더라도, 이는 우리가 LLM을 사용하는 방법을 몰라서인지 아니면 LLM이 근본적으로 계획할 수 없기 때문인지 확실하지 않습니다.

**"계획은 본질적으로 탐색 문제(search problem)입니다."** 목표로 가는 여러 경로를 탐색하고, 각 경로의 결과(보상)를 예측하며, 가장 유망한 결과를 가진 경로를 선택합니다. 종종, 목표로 가는 경로가 없다는 결론에 도달할 수 있습니다.

탐색은 종종 **백트래킹(backtracking)**을 요구합니다. 예를 들어, 특정 단계에서 두 가지 행동 옵션(A와 B)이 있다고 가정해봅시다. A를 실행한 후, 그 상태가 유망하지 않다는 것을 알게 된다면, 이전 단계로 돌아가 B를 선택해야 합니다.

어떤 사람들은 오토레그레시브 모델이 오직 몇 가지의 고정된 행동만 생성할 수 있다고 주장합니다. 이는 새로운 대안을 생성하지 못한다는 의미로, 오토레그레시브 모델은 계획할 수 없다는 결론을 내리게 합니다. 하지만 반드시 그런 것은 아닙니다. 모델이 A 경로를 실행한 후, 해당 경로가 유효하지 않다는 결정을 내리면, B를 다시 실행하거나 처음부터 다시 시작하여 다른 경로를 선택할 수 있습니다.

LLM이 형편없는 플래너인 이유는 계획에 필요한 도구가 제공되지 않았기 때문일 수도 있습니다. 계획을 세우려면, 가능한 행동뿐만 아니라 **각 행동의 잠재적인 결과(potential outcome)**도 알아야 합니다. 간단한 예로, 산을 오르려 한다고 가정해봅시다. 가능한 행동은 오른쪽으로 돌기, 왼쪽으로 돌기, 뒤로 돌기, 직진하기가 있습니다. 그러나 오른쪽으로 도는 것이 절벽에서 떨어지는 결과를 초래한다면, 이 행동을 고려하지 않는 것이 좋습니다. 기술적으로는, 행동이 상태를 한 단계에서 다른 단계로 가져가며, 다음 행동을 결정하기 위해 결과 상태를 아는 것이 필요합니다.

따라서 모델에게 인기 있는 **체인 오브 사고(chain-of-thought)**와 같은 동작의 단순한 순서를 생성하라고 요청하는 것만으로는 충분하지 않습니다. Hao et al.(2023)의 논문 **"Reasoning with Language Model is Planning with World Model"**은 세계에 대한 풍부한 정보를 포함하는 LLM이 각 행동의 결과를 예측하고 더 나은 계획을 생성할 수 있다고 주장합니다.

심지어 AI가 계획을 할 수 없더라도, 여전히 계획의 일부로 포함될 수 있습니다. 검색 도구와 상태 추적 시스템을 LLM에 통합하여 계획을 돕는 것이 가능할 수 있습니다.

---

>**기반 모델(FM) 대 강화 학습(RL) 플래너 비교**
>
>에이전트(agent)는 강화 학습(RL)의 핵심 개념으로, **"지속적으로 누적 보상을 극대화하기 위해 동적 환경에서 행동을 취하는 방식"**으로 정의됩니다.
>
>RL 에이전트와 FM 에이전트는 많은 면에서 유사합니다. 둘 다 환경과 가능한 행동에 의해 특징지어집니다. 주요 차이점은 계획자가 작업하는 방식입니다. RL 에이전트에서 계획자는 RL 알고리즘에 의해 훈련됩니다. 이 RL 계획자를 훈련하는 데는 많은 시간과 자원이 필요할 수 있습니다. 반면 FM 에이전트에서는 모델 자체가 계획자입니다. 이 모델은 미세 조정되거나 기본 기능으로 계획 능력을 통합할 수 있으며, 일반적으로 적은 시간과 자원을 필요로 합니다.
>
>그러나 FM 에이전트가 RL 알고리즘을 통합하여 성능을 향상시키는 것을 막을 이유는 없습니다. 장기적으로는 FM 에이전트와 RL 에이전트가 합쳐질 가능성이 있다고 생각됩니다.

---

**계획 생성 (Plan generation)**

모델을 계획 생성기로 전환하는 가장 간단한 방법은 프롬프트 엔지니어링을 사용하는 것입니다. Kitty Vogue의 고객이 제품에 대해 더 많이 알 수 있도록 돕는 에이전트를 만들고 싶다고 상상해보세요. 이 에이전트에게 제품을 가격별로 검색, 상위 제품 검색, 제품 정보 검색과 같은 세 가지 외부 도구에 대한 접근 권한을 부여합니다. 다음은 계획 생성을 위한 프롬프트 예시입니다. 이 프롬프트는 설명 목적으로만 제공되며, 실제 프로덕션 프롬프트는 더 복잡할 가능성이 높습니다:

**SYSTEM PROMPT**  
주어진 작업을 해결할 계획을 제안하세요. 다음 5가지 작업에 접근할 수 있습니다:  
1. `get_today_date()`  
2. `fetch_top_products(start_date, end_date, num_products)`  
3. `fetch_product_info(product_name)`  
4. `generate_query(task_history, tool_output)`  
5. `generate_response(query)`  

계획은 유효한 작업의 순서여야 합니다.

**예시**  
작업: "Fruity Fedora에 대해 알려주세요."  
계획: `[fetch_product_info, generate_query, generate_response]`  

작업: "지난주 가장 잘 팔린 제품은 무엇이었나요?"  
계획: `[fetch_top_products, generate_query, generate_response]`  

작업: {사용자 입력}  
계획:  

이 예시에서 주목할 두 가지 사항이 있습니다:  
- 여기에서 사용된 계획 형식(에이전트에 의해 매개변수가 추론되는 함수 목록)은 에이전트 제어 흐름을 구조화하는 여러 방법 중 하나일 뿐입니다.
- `generate_query` 함수는 작업의 현재 기록과 가장 최근의 도구 출력을 활용하여 응답 생성기로 입력할 쿼리를 생성합니다. 각 단계의 도구 출력은 작업 기록에 추가됩니다.

사용자 입력 "지난주 가장 잘 팔린 제품의 가격은 얼마였나요?"를 기준으로 생성된 계획은 다음과 같을 수 있습니다:  
1. `get_time()`  
2. `fetch_top_products()`  
3. `fetch_product_info()`  
4. `generate_query()`  
5. `generate_response()`  

"각 함수에 필요한 매개변수는 무엇인지?" **궁금할 수도 있습니다:**. 각 단계에서 매개변수는 이전 도구 출력에서 추출되기 때문에 사전에 정확히 예측하기 어렵습니다. 첫 번째 단계인 `get_time()`이 "2030-09-13"을 출력하면, 에이전트는 다음 단계의 매개변수가 다음과 같이 호출되어야 한다고 추론할 수 있습니다:  

```
retrieve_top_products(
    start_date="2030-09-07",
    end_date="2030-09-13",
    num_products=1
)
```

종종 함수에 필요한 정확한 매개변수 값을 결정하기에 충분한 정보가 제공되지 않습니다. 예를 들어, 사용자가 "가장 잘 팔리는 제품들의 평균 가격은 얼마인가요?"라고 물을 경우, 다음 질문들에 대한 답변이 불명확합니다:

- 사용자가 보고 싶어하는 가장 잘 팔리는 제품의 개수는 몇 개인가요?  
- 사용자는 지난주, 지난달, 아니면 전체 기간의 가장 잘 팔리는 제품을 원하나요?

이는 모델이 종종 추측을 해야 하며, 이러한 추측이 틀릴 수 있음을 의미합니다.

행동 순서와 관련 매개변수가 AI 모델에 의해 생성되기 때문에, 잘못된 함수 호출이나 잘못된 매개변수를 사용한 유효한 함수 호출과 같은 오류가 발생할 수 있습니다. 일반적으로 모델의 성능을 향상시키기 위한 기술을 사용하여 모델의 계획 능력을 개선할 수 있습니다.

**에이전트가 계획을 더 잘 수행하도록 만드는 몇 가지 접근 방식:**  
- 더 많은 예시가 포함된 더 나은 시스템 프롬프트 작성.  
- 도구와 그 매개변수에 대한 더 나은 설명 제공.  
- 복잡한 함수를 두 개의 더 간단한 함수로 리팩토링하는 등 함수 자체를 더 간단하게 다시 작성.  
- 더 강력한 모델 사용. 일반적으로, 강력한 모델이 계획을 더 잘 수행합니다.  
- 계획 생성을 위해 모델을 미세 조정(Finetune).  

**함수 호출 (Function calling)**

많은 모델 제공자는 모델을 위한 도구 사용 기능을 제공하여 모델을 에이전트로 효과적으로 변환합니다. 도구는 하나의 함수입니다. 도구 호출은 종종 **"함수 호출(function calling)"**이라고 불립니다. 모델 API는 각각 다르게 작동하지만, 일반적으로 함수 호출은 다음과 같이 작동합니다:

1. **도구 목록 작성(Create a tool inventory):**  
   모델이 사용할 수 있는 도구를 선언합니다. 각 도구는 실행 진입점(예: 함수 이름), 매개변수, 그리고 문서화(예: 함수의 기능 및 필요한 매개변수)로 설명됩니다.

2. **에이전트가 사용할 도구 지정(Specify what tools the agent can use):**  
   서로 다른 쿼리는 서로 다른 도구를 필요로 할 수 있으므로, 많은 API는 쿼리당 선언된 도구 목록을 지정할 수 있도록 합니다. 일부 API는 다음과 같은 설정을 통해 도구 사용을 더욱 제어할 수 있도록 허용합니다:
   - **필수(required):**  
     모델이 최소한 하나의 도구를 사용해야 합니다.
   - **없음(none):**  
     모델이 어떤 도구도 사용해서는 안 됩니다.
   - **자동(auto):**  
     모델이 사용할 도구를 스스로 결정합니다.

함수 호출은 **Figure 6-10**에 예시로 나와 있습니다. 이는 여러 API를 대표하도록 의사 코드로 작성되었습니다. 특정 API를 사용하려면 해당 문서를 참조하십시오.

<img src="./images/fig_06_10.png" width="800">

주어진 쿼리에 대해, **Figure 6-10**에 정의된 에이전트는 자동으로 사용할 도구와 해당 매개변수를 생성합니다. 함수 호출 API는 유효한 함수만 생성되도록 보장하지만, 매개변수 값의 정확성을 보장할 수는 없습니다.

예를 들어, 사용자가 "40파운드는 몇 킬로그램인가요?"라고 질문했을 때, 에이전트는 `lbs_to_kg_tool`이 필요하다고 판단하며, 매개변수 값으로 40을 설정할 수 있습니다. 에이전트의 응답은 다음과 같이 나타날 수 있습니다:

```python
response = ModelResponse(
    finish_reason='tool_calls',
    message=chat.Message(
        content=None,
        role='assistant',
        tool_calls=[
            ToolCall(
                function=Function(
                    arguments='{"lbs":40}',
                    name='lbs_to_kg'
                ),
                type='function'
            )
        ]
    )
)
```

이 응답에서, `lbs_to_kg(lbs=40)` 함수를 호출하고 그 출력을 사용하여 사용자에게 응답을 생성할 수 있습니다.

---

>**TIP**  
>
>에이전트와 작업할 때, 시스템이 각 함수 호출에 사용한 매개변수 값을 보고하도록 요청하세요. 이러한 값을 점검하여 정확성을 확인하세요.

---

**계획의 세분화 (Planning granularity)**

계획은 작업을 완료하기 위해 필요한 단계를 개략적으로 나타내는 로드맵입니다. 로드맵은 다양한 수준의 세분화를 가질 수 있습니다. 예를 들어, 1년 계획은 분기별 계획보다 더 고수준이고, 월별 계획은 주별 계획보다 더 고수준입니다.

**계획과 실행 간에는 트레이드오프가 있습니다.** 세부적인 계획은 생성하기 어렵지만 실행하기 쉽습니다. 반대로, 고수준의 계획은 생성하기는 쉽지만 실행하기 어려울 수 있습니다. 이러한 절충안을 해결하는 접근 방식은 계층적으로 계획을 세우는 것입니다. 먼저, 분기별 계획과 같은 고수준 계획을 생성합니다. 그런 다음, 각 분기를 위해 동일한 또는 다른 플래너를 사용하여 월별 계획을 생성합니다.

지금까지의 생성된 계획 예시들은 매우 세분화된 함수 이름을 사용합니다. 이 접근 방식의 문제는 에이전트의 도구 목록이 시간이 지나면서 변경될 수 있다는 점입니다. 예를 들어, 현재 날짜를 가져오는 함수인 `get_time()`이 `get_current_time()`으로 이름이 변경된다면, 프롬프트와 모든 예시를 업데이트해야 합니다. 정확한 함수 이름을 사용하는 것은 도구 API가 다른 경우에 동일한 플래너를 재사용하기 어렵게 만듭니다.

이전에 특정 도구 목록을 기반으로 계획 생성을 위해 모델을 미세 조정했다면, 새로운 도구 목록에 대해 모델을 다시 미세 조정해야 할 수도 있습니다.

이 문제를 피하기 위해, 계획은 도메인별 함수 이름보다 더 높은 수준의 자연어를 사용하여 생성될 수도 있습니다. 예를 들어, "지난주 가장 많이 팔린 제품의 가격은 얼마입니까?"라는 쿼리에 대해, 에이전트는 다음과 같은 계획을 출력하도록 지시받을 수 있습니다:

1. 현재 날짜 가져오기  
2. 지난주 가장 많이 팔린 제품 검색  
3. 제품 정보 검색  
4. 쿼리 생성  
5. 응답 생성  

자연어를 사용하는 것은 계획 생성기가 도구 API 변경에 더 잘 적응할 수 있도록 도와줍니다. 모델이 주로 자연어로 학습된 경우, 자연어 계획을 이해하고 생성하는 데 더 뛰어나며 환각(hallucination)이 발생할 가능성이 적습니다.

이 접근 방식의 단점은 각 자연어 행동을 실행 가능한 명령어로 번역하는 번역기가 필요하다는 점입니다. 하지만 번역은 계획 수립보다 훨씬 간단한 작업이며, 환각 위험이 낮은 더 약한 모델에서도 수행할 수 있습니다.

**복잡한 계획 (Complex plans)**

지금까지의 계획 예시는 순차적이었습니다. 계획에서 다음 작업은 항상 이전 작업이 완료된 후에 실행됩니다. 작업 실행 순서를 제어하는 방식을 **제어 흐름 (control flow)**라고 합니다. 순차적 형식은 제어 흐름의 한 가지 유형일 뿐입니다. 병렬 처리(parallel), if 문, for 루프 등 다른 제어 흐름 유형도 있습니다. 아래는 순차적 계획을 포함한 각 제어 흐름의 개요입니다:

- **순차적 (Sequential)**  
  작업 A가 완료된 후 작업 B를 실행합니다. 이는 작업 B가 작업 A에 의존하기 때문일 수 있습니다. 예를 들어, SQL 쿼리는 자연어 입력에서 변환된 후에만 실행될 수 있습니다.

- **병렬 처리 (Parallel)**  
  작업 A와 작업 B를 동시에 실행합니다. 예를 들어, "100달러 이하의 가장 많이 팔린 제품을 찾아라"라는 쿼리의 경우, 에이전트는 먼저 상위 100개의 가장 많이 팔린 제품을 검색한 다음, 각 제품의 가격을 검색할 수 있습니다.

- **if 문**  
  이전 단계의 출력에 따라 작업 B 또는 작업 C를 실행합니다. 예를 들어, 에이전트가 NVIDIA의 수익 보고서를 확인한 다음, 이 보고서를 기반으로 NVIDIA 주식을 매도하거나 매수할 수 있습니다.

- **for 루프**  
  특정 조건이 충족될 때까지 작업 A를 반복 실행합니다. 예를 들어, 소수가 나올 때까지 임의의 숫자를 계속 생성합니다.

이러한 다양한 제어 흐름은 **Figure 6-11**에 시각화되어 있습니다.

<img src="./images/fig_06_11.png" width="800">

전통적인 소프트웨어 엔지니어링에서 제어 흐름의 조건은 정확합니다. 그러나 AI 기반 에이전트에서는 AI 모델이 제어 흐름을 결정합니다. 비순차적 제어 흐름이 포함된 계획은 실행 가능한 명령으로 변환하기 더 어렵습니다.

에이전트 프레임워크를 평가할 때, 어떤 제어 흐름을 지원하는지 확인하세요. 예를 들어, 시스템이 여러 웹사이트를 탐색해야 할 때, 동시에 탐색할 수 있습니까? 병렬 실행은 사용자에게 인지되는 지연 시간을 크게 줄일 수 있습니다.

---

**반성 및 오류 수정**

최고의 계획이라 하더라도 성공 가능성을 극대화하기 위해 지속적으로 평가되고 조정되어야 합니다. 반성이 에이전트 작동에 반드시 필요한 것은 아니지만, 성공을 위해 필수적입니다.

반성은 작업 과정에서 다음과 같은 여러 시점에서 유용하게 사용될 수 있습니다:

- 사용자 요청을 검토한 후, 요청이 실행 가능한지 평가.
- 초기 계획을 생성한 후, 계획이 타당한지 평가.
- 각 실행 단계를 완료한 후, 진행 방향이 올바른지 평가.
- 전체 계획이 완료된 후, 작업이 제대로 수행되었는지 평가.

반성과 오류 수정은 밀접하게 연결된 두 가지 메커니즘입니다. 반성은 통찰력을 생성하여 수정이 필요한 오류를 발견할 수 있도록 합니다.

반성은 자기비판적 프롬프트를 사용하는 동일한 에이전트를 통해 수행할 수도 있고, 결과마다 구체적인 점수를 출력하는 특화된 스코어링 모델과 같은 별도의 구성 요소를 통해 수행할 수도 있습니다.

ReAct(Reasoning + Acting, Yao et al., 2022) 프레임워크는 추론과 행동을 교차적으로 수행하는 에이전트의 일반적인 패턴으로 처음 제안되었습니다. Yao et al.은 "추론"이라는 용어를 계획 및 반성을 포함하는 포괄적인 개념으로 사용했습니다. 각 단계에서 에이전트는 사고(계획)를 설명하고, 행동을 실행하고, 관찰 결과를 분석(반성)한 후, 작업이 완료되었다고 반성이 판단할 때까지 이를 반복합니다. 에이전트는 일반적으로 예제를 사용하여 다음 형식으로 출력을 생성하도록 프롬프트됩니다:

- **생각 1:** ...
- **행동 1:** ...
- **관찰 1:** ...

(... 작업이 완료되었다고 반성이 판단할 때까지 계속 ...)

- **생각 N:** ...
- **행동 N: Finish** [쿼리에 대한 응답]

**Figure 6-12**는 HotpotQA (Yang et al., 2018)에 대한 질문에 ReAct 프레임워크를 따르는 에이전트의 예제를 보여줍니다. 이는 다중 홉 질문 응답에 대한 벤치마크입니다.

<img src="./images/fig_06_12.png" width="800">

다중 에이전트 환경에서 반성을 구현할 수도 있습니다: 하나의 에이전트가 계획을 수립하고 행동을 실행하는 동안 다른 에이전트가 각 단계 이후 또는 일정 단계 후 결과를 평가합니다.

만약 에이전트의 응답이 작업을 완료하지 못했다면, 에이전트에게 실패 이유와 개선 방법에 대해 반성하도록 프롬프트를 제공할 수 있습니다. 이 제안을 바탕으로 에이전트는 새로운 계획을 생성합니다. 이를 통해 에이전트가 실수를 통해 학습할 수 있습니다. 예를 들어, 코딩 생성 작업에서 평가자는 생성된 코드가 테스트 케이스의 1/3만 통과한다고 평가할 수 있습니다. 그런 다음 에이

전트는 모든 숫자가 음수인 배열을 고려하지 않았기 때문에 실패했다고 반성하며, 이를 기반으로 새로운 코드를 생성하면서 모든 음수 배열을 포함하도록 개선할 수 있습니다.

이 접근법은 **Reflexion (Shinn et al., 2023)**에서 사용되었습니다. 이 프레임워크에서는 결과를 평가하는 평가자와 잘못된 점을 분석하는 자기 반성 모듈로 나뉘어 있습니다. **Figure 6-13**은 행동 중인 Reflexion 에이전트의 예제를 보여줍니다. 저자들은 "trajectory(궤적)"라는 용어를 계획을 지칭하기 위해 사용했습니다. 각 단계에서 평가와 자기 반성을 통해 에이전트는 새로운 궤적을 제안합니다.

<img src="./images/fig_06_13.png" width="800">

계획 생성과 비교했을 때, 반성은 상대적으로 쉽게 구현할 수 있으며 놀라울 정도로 좋은 성능을 제공할 수 있습니다. 그러나 이 접근법의 단점은 지연(latency)과 비용(cost)입니다. 생각, 관찰 및 때로는 행동이 많은 토큰을 생성해야 하며, 이는 특히 여러 중간 단계가 필요한 작업에서 비용과 사용자 지연 인식을 증가시킵니다. 포맷을 따르기 위해 필요한 토큰 수를 줄이기 위해 ReAct 및 Reflexion 작성자들은 출력 예제를 풍부하게 활용했습니다. 이는 컴퓨팅 토큰을 줄이고, 다른 정보에 대해 사용할 수 있는 맥락 공간을 늘립니다.

--- 

**도구 선택**

도구는 작업 성공에 중요한 역할을 하기 때문에, 도구 선택에는 신중한 고려가 필요합니다. 에이전트에 제공할 도구는 환경과 작업에 따라 다를 뿐만 아니라, 에이전트를 구동하는 AI 모델에도 의존합니다.

도구를 최적의 세트로 선택하기 위한 완벽한 가이드라인은 없습니다. 에이전트 관련 문헌에는 다양한 도구 목록이 포함되어 있습니다. 예를 들어, **Toolformer(Schick et al., 2023)**는 GPT-J를 미세 조정하여 5가지 도구를 학습하도록 했으며, **Chameleon(Lu et al., 2023)**은 13개의 도구를 사용합니다. 반면에, **Gorilla(Patil et al., 2023)**는 1,645개의 API 중 올바른 API 호출을 선택하도록 에이전트를 프롬프트 기반으로 학습시켰습니다.

도구가 많아질수록 에이전트의 능력은 증가합니다. 하지만 도구가 많아질수록 이를 효율적으로 사용하는 것도 더 어려워집니다. 이는 인간이 많은 도구 세트를 마스터하기 어려운 것과 유사합니다. 도구를 추가하는 것은 도구 설명을 늘리며, 이는 모델의 맥락을 초과할 가능성을 높입니다.

AI 응용 프로그램을 구축하는 데 있어 다른 많은 결정과 마찬가지로, 도구 선택은 실험과 분석을 필요로 합니다. 다음은 이를 결정하는 데 도움이 될 몇 가지 방법입니다:

- 다양한 도구 세트를 가진 에이전트의 성능을 비교하세요.
- 특정 도구를 제거했을 때 에이전트 성능이 얼마나 저하되는지 확인하는 실험을 하세요. 성능 저하가 적다면 해당 도구를 제거하세요.
- 에이전트가 자주 실수를 하는 도구를 확인하세요. 에이전트가 배우기 어려운 도구라면, 적절한 프롬프팅과 미세 조정을 거쳐도 학습하지 못한다면, 해당 도구를 교체하세요.
- 도구 사용 빈도를 살펴보고 가장 많이 사용되는 도구와 가장 적게 사용되는 도구를 확인하세요. **Figure 6-14**는 Chameleon(Lu et al., 2023)에서 GPT-4와 ChatGPT의 도구 사용 패턴 차이를 보여줍니다.

<img src="./images/fig_06_14.png" width="800">

**Lu et al. (2023)**의 실험은 다음 두 가지 사항을 강조합니다:

1. 서로 다른 작업에는 서로 다른 도구가 필요합니다. 예를 들어, **ScienceQA**라는 과학 질문 답변 작업은 표 형식 수학 문제 해결 작업(TabMWP)보다 지식 검색 도구를 훨씬 더 많이 사용합니다.
2. 서로 다른 모델은 서로 다른 도구 선호도를 보입니다. 예를 들어, GPT-4는 ChatGPT보다 더 넓은 범위의 도구를 선택하는 경향이 있습니다. ChatGPT는 이미지 캡셔닝을 선호하는 반면, GPT-4는 지식 검색을 선호합니다.

>**팁:** 
>
>에이전트 프레임워크를 평가할 때, 어떤 플래너와 도구를 지원하는지 평가하세요. 프레임워크에 따라 도구 카테고리 초점이 다를 수 있습니다. 예를 들어, **AutoGPT**는 소셜 미디어 API(Reddit, X, Wikipedia)에 집중하는 반면, **Composio**는 기업 API(Google Apps, GitHub, Slack)에 중점을 둡니다. 요구 사항은 시간이 지나면서 변경될 가능성이 있으므로, 에이전트가 새로운 도구를 통합하기 얼마나 쉬운지 평가하세요.

---

인간은 주어진 도구만 사용하는 것이 아니라 점점 더 강력한 도구를 단순한 도구로부터 만들어내며 생산성을 높입니다. AI도 초기 도구로부터 새로운 도구를 생성할 수 있을까요?

**Chameleon (Lu et al., 2023)**은 도구 전환(tool transition)의 연구를 제안합니다: 도구 X를 사용한 후, 에이전트가 도구 Y를 호출할 가능성은 얼마나 될까요? **Figure 6-15**는 도구 전환의 예를 보여줍니다. 두 도구가 자주 함께 사용된다면, 이를 더 큰 도구로 통합할 수 있습니다. 에이전트가 이러한 정보를 인식하고 있다면, 초기 도구를 결합하여 점점 더 복잡한 도구를 지속적으로 구축할 수 있습니다.

<img src="./images/fig_06_15.png" width="800">

**Vogager(Wang et al., 2023)**는 에이전트가 후속 작업을 위해 획득한 새로운 기술(도구)을 기록하는 스킬 매니저를 제안합니다. 각 기술은 코딩 프로그램입니다. 스킬 매니저는 새로운 기술이 유용하다고 판단되면(예: 작업 완료에 성공적으로 도움이 되는 경우), 이를 스킬 라이브러리에 추가합니다(개념적으로는 도구 목록과 유사합니다). 이후 다른 작업에서 이를 검색하여 사용할 수 있습니다.

앞서 언급한 바와 같이, 에이전트의 성공은 환경 내 도구 인벤토리와 계획 능력에 달려 있습니다. 두 측면 중 하나라도 실패하면 에이전트는 작업을 실패할 수 있습니다. 다음 섹션에서는 서로 다른 실패 모드와 이를 평가하는 방법을 논의할 것입니다.

---


### **에이전트 실패 모드 및 평가**

평가는 실패를 감지하는 데 중점을 둡니다. 에이전트가 수행하는 작업이 복잡할수록 더 많은 실패 지점이 발생할 가능성이 있습니다. 3장과 4장에서 논의된 AI 애플리케이션에 공통된 실패 모드 외에도, 에이전트는 계획, 도구 실행 및 효율성에서 고유한 실패를 경험할 수 있습니다. 일부 실패는 다른 것들보다 감지하기가 더 쉽습니다.

에이전트를 평가하려면, 실패 모드를 식별하고 각 실패 모드가 얼마나 자주 발생하는지 측정하세요.

이러한 다양한 실패 모드를 설명하기 위해 GitHub 저장소에서 확인할 수 있는 간단한 벤치마크를 만들었습니다. 또한 **Berkeley Function Calling Leaderboard**, **AgentOps 평가 도구**, **TravelPlanner 벤치마크**와 같은 에이전트 벤치마크 및 순위표도 있습니다.

---

**계획 실패**

계획은 어려우며 여러 방식으로 실패할 수 있습니다. 계획 실패의 가장 일반적인 형태는 도구 사용 실패입니다. 에이전트는 다음과 같은 하나 이상의 오류가 포함된 계획을 생성할 수 있습니다:

- **잘못된 도구:**  
  예를 들어, `bing_search`를 포함한 계획을 생성하지만, `bing_search`는 에이전트의 도구 목록에 없습니다.

- **유효한 도구, 잘못된 매개변수:**  
  예를 들어, `lbs_to_kg`를 두 개의 매개변수로 호출합니다. `lbs_to_kg`는 도구 목록에 있지만, 한 개의 매개변수만 필요합니다.

- **유효한 도구, 잘못된 매개변수 값:**  
  예를 들어, `lbs_to_kg`를 한 개의 매개변수(`lbs`)로 호출하지만, 매개변수 값이 120이어야 할 자리에 100을 사용합니다.

계획 실패의 또 다른 모드는 목표 실패입니다. 에이전트가 목표를 달성하지 못하는 경우입니다. 이는 계획이 작업을 해결하지 못하거나 제한 조건을 준수하지 않은 경우에 발생합니다. 이를 설명하기 위해, 에이전트에게 샌프란시스코에서 하노이까지 여행을 계획하도록 요청하면서 예산을 $5,000로 설정했다고 가정해보세요. 에이전트는 샌프란시스코에서 호치민시로 가는 여행을 계획하거나, 예산을 초과하는 샌프란시스코에서 하노이까지의 여행을 계획할 수 있습니다.

에이전트 평가에서 종종 간과되는 일반적인 제한 조건은 **시간**입니다. 많은 경우, 에이전트가 작업을 완료하는 데 걸리는 시간은 중요하지 않다고 생각할 수 있습니다. 그러나 많은 경우, 에이전트가 작업을 완료하는 시간은 중요합니다. 예를 들어, 에이전트에게 보조금 제안서를 준비하도록 요청했는데 마감일 이후에 작업을 완료하면 이는 유용하지 않습니다.

계획 실패의 흥미로운 형태는 **반성 중 오류**에서 비롯됩니다. 에이전트는 작업을 완료하지 않았음에도 불구하고 완료했다고 확신할 수 있습니다. 예를 들어, 에이전트에게 30개의 호텔 방에 50명을 배정하라고 요청했는데, 40명만 배정하고 작업이 완료되었다고 주장할 수 있습니다.

---

**계획 실패 평가 방법**

계획 실패를 평가하는 한 가지 방법은 각 예제가 튜플(작업, 도구 목록)로 구성된 계획 데이터셋을 만드는 것입니다. 그런 다음 K개의 계획을 생성하도록 에이전트를 사용하고 다음 메트릭을 계산합니다:

1. 생성된 모든 계획 중, 유효한 계획은 몇 개입니까?  
2. 특정 작업에 대해, 유효한 계획을 생성하기 위해 에이전트가 평균적으로 몇 개의 계획을 생성해야 합니까?  
3. 모든 도구 호출 중, 유효한 도구 호출은 몇 개입니까?  
4. 잘못된 도구는 얼마나 자주 호출됩니까?  
5. 잘못된 매개변수로 유효한 도구가 호출된 빈도는 얼마나 됩니까?  
6. 잘못된 매개변수 값으로 유효한 도구가 호출된 빈도는 얼마나 됩니까?

에이전트의 출력 패턴을 분석하세요. 에이전트가 어떤 유형의 작업에서 더 자주 실패합니까? 이에 대한 가설이 있습니까? 모델이 자주 실수하는 도구는 무엇입니까? 일부 도구는 에이전트가 사용하기에 더 어려울 수 있습니다. 더 나은 프롬프팅, 예제 제공, 미세 조정을 통해 에이전트가 어려운 도구를 사용하는 능력을 개선할 수 있습니다. 모든 방법이 실패할 경우, 해당 도구를 더 쉽게 사용할 수 있는 도구로 교체하는 것을 고려할 수 있습니다.

---

**도구 실패**

도구 실패는 올바른 도구가 사용되었지만 도구 출력이 잘못된 경우에 발생합니다. 한 가지 실패 모드는 도구가 단순히 잘못된 출력을 제공할 때입니다. 예를 들어, 이미지 캡셔너가 잘못된 설명을 반환하거나 SQL 쿼리 생성기가 잘못된 SQL 쿼리를 반환하는 경우가 있습니다.

에이전트가 고수준 계획만 생성하고, 번역 모듈이 각 계획된 작업을 실행 가능한 명령으로 변환하는 경우, 번역 오류로 인해 실패가 발생할 수 있습니다.

또한 에이전트가 작업에 적합한 도구에 접근할 수 없을 때도 도구 실패가 발생할 수 있습니다. 명확한 예로, 인터넷에서 현재 주가를 검색해야 하는 작업이 있을 때, 에이전트가 인터넷에 접근할 수 없는 경우를 들 수 있습니다.

도구 실패는 도구에 따라 달라집니다. 각 도구는 독립적으로 테스트되어야 합니다. 항상 각 도구 호출과 그 출력을 출력하여 이를 점검하고 평가하세요. 번역기가 있다면, 이를 평가하기 위한 벤치마크를 생성하세요.

누락된 도구 실패를 감지하려면 어떤 도구가 사용되어야 하는지에 대한 이해가 필요합니다. 에이전트가 특정 도메인에서 자주 실패한다면, 이는 해당 도메인에 대한 도구가 부족하기 때문일 수 있습니다. 도메인 전문가와 협력하여 그들이 사용할 도구를 관찰하세요.

---

**효율성**

에이전트는 올바른 도구를 사용하여 유효한 계획을 생성하여 작업을 완료할 수 있지만, 비효율적일 수 있습니다. 에이전트의 효율성을 평가하기 위해 추적할 수 있는 몇 가지 요소는 다음과 같습니다:

- 작업을 완료하기 위해 에이전트가 평균적으로 몇 단계가 필요한가요?
- 작업을 완료하는 데 에이전트가 평균적으로 얼마나 많은 비용이 드나요?
- 각 작업은 일반적으로 얼마나 걸리나요? 특히 시간이 많이 걸리거나 비용이 많이 드는 작업이 있나요?

이러한 메트릭을 기준선과 비교할 수 있습니다. 기준선은 다른 에이전트나 인간 운영자가 될 수 있습니다. AI 에이전트를 인간과 비교할 때, 인간과 AI는 매우 다른 작업 방식이 있다는 점을 명심해야 합니다. 인간에게 비효율적이더라도 AI에게는 효율적일 수 있고, 그 반대도 마찬가지입니다. 예를 들어, 100개의 웹페이지를 방문하는 것은 한 번에 하나의 페이지를 방문할 수 있는 인간에게는 비효율적일 수 있지만, 모든 웹페이지를 동시에 방문할 수 있는 AI 에이전트에게는 사소한 일일 수 있습니다.

이 장에서는 RAG 및 에이전트 시스템이 어떻게 작동하는지에 대해 자세히 논의했습니다. 두 가지 패턴 모두 모델의 컨텍스트 한계를 초과하는 정보를 다룹니다. 메모리 시스템은 정보를 처리하는 모델의 컨텍스트를 보완하여 에이전트의 역량을 상당히 향상시킬 수 있습니다. 이제 메모리 시스템이 어떻게 작동하는지 살펴보겠습니다.

---

## **메모리**

메모리는 모델이 정보를 유지하고 활용할 수 있도록 하는 메커니즘을 말합니다. 메모리 시스템은 RAG와 같은 지식 중심 응용 프로그램이나 에이전트와 같은 다단계 응용 프로그램에 특히 유용합니다. RAG 시스템은 여러 번의 반복을 통해 정보를 검색하면서 증가하는 확장된 컨텍스트에 메모리를 의존합니다. 에이전트 시스템은 명령어, 예제, 컨텍스트, 도구 목록, 계획, 도구 출력, 반성 등을 저장하기 위해 메모리가 필요합니다. RAG와 에이전트가 메모리에 더 큰 부담을 주더라도, 정보를 유지해야 하는 AI 응용 프로그램에 메모리 시스템은 매우 유익합니다.

AI 모델은 일반적으로 다음 세 가지 주요 메모리 메커니즘을 가지고 있습니다:

- **내부 지식(Internal knowledge)**  
  모델 자체가 메모리 메커니즘이며, 훈련 데이터에서 얻은 지식을 유지합니다. 이 지식은 모델의 **내부 지식**입니다. 모델의 내부 지식은 모델 자체가 업데이트되지 않는 한 변경되지 않습니다. 모델은 모든 쿼리에서 이 지식에 접근할 수 있습니다.
- **단기 메모리(Short-term memory)**  
  모델의 컨텍스트는 메모리 메커니즘입니다. 대화의 이전 메시지를 모델의 컨텍스트에 추가하면, 모델은 이를 활용하여 이후 응답을 생성할 수 있습니다. 모델의 컨텍스트는 단기 메모리로 간주될 수 있으며, 이는 작업(쿼리) 간에 유지되지 않기 때문입니다. 단기 메모리는 접근 속도가 빠르지만, 용량이 제한적입니다. 따라서 일반적으로 현재 작업에서 가장 중요한 정보를 저장하는 데 사용됩니다.
- **장기 메모리(Long-term memory)**  
  모델이 검색을 통해 접근할 수 있는 외부 데이터 소스, 예를 들어 RAG 시스템에서의 데이터 소스는 메모리 메커니즘으로 간주됩니다. 이는 모델의 **장기 메모리**로 간주될 수 있으며, 여러 작업에 걸쳐 유지될 수 있습니다. 모델의 내부 지식과 달리, 장기 메모리의 정보는 모델을 업데이트하지 않고 삭제할 수 있습니다.

인간도 이와 유사한 메모리 메커니즘에 접근할 수 있습니다. 예를 들어, 숨을 쉬는 방법을 잊지 않는 것은 내부 지식에 해당합니다. 심각한 문제에 직면하지 않는 한 호흡 방법을 잊지 않습니다. 단기 메모리는 방금 만난 사람의 이름과 같이 즉각적으로 관련된 정보를 포함합니다. 장기 메모리는 책, 컴퓨터, 메모와 같은 자료로 보완됩니다.

어떤 메모리 메커니즘을 사용할지는 정보 사용 빈도에 따라 달라집니다. 모든 작업에 필수적인 정보는 모델의 내부 지식에 통합되어야 합니다(훈련이나 미세 조정을 통해). 드물게 필요한 정보는 장기 메모리에 있어야 합니다. 단기 메모리는 즉각적이고 특정한 맥락의 정보를 저장하는 데 사용됩니다. 이 세 가지 메모리 메커니즘은 **Figure 6-16**에 시각적으로 설명되어 있습니다.

<img src="./images/fig_06_16.png" width="800">

메모리는 인간이 작업을 수행하는 데 필수적이며, AI 애플리케이션에서도 점점 중요한 요소로 자리 잡고 있습니다. AI 모델용 메모리 관리 도구가 다양하게 개발되었으며, 많은 모델 제공자들이 메모리 시스템을 통합했습니다. AI 모델에 메모리 시스템을 추가하면 많은 이점이 있습니다. 다음은 몇 가지 주요 이점입니다:

- **세션 내에서 정보 초과 관리**  
  작업 수행 중 에이전트는 많은 새로운 정보를 습득하게 되며, 이는 에이전트의 최대 컨텍스트 길이를 초과할 수 있습니다. 초과된 정보는 장기 메모리에 저장되어 관리될 수 있습니다.

- **세션 간 정보 유지**  
  코치의 조언을 받을 때마다 자신의 모든 배경을 설명해야 한다면 비효율적일 것입니다. AI 비서가 사용자의 선호도를 잊어버린다면 불편할 것입니다. 대화 기록에 접근함으로써, 에이전트는 행동을 사용자에 맞게 조정할 수 있습니다. 예를 들어, 책 추천을 요청할 때, 에이전트가 이전에 당신이 『삼체(The Three-Body Problem)』를 좋아했다고 기억한다면, 유사한 책을 추천할 수 있습니다.

- **모델의 일관성 향상**  
  주관적인 질문, 예를 들어 "1부터 5까지 농담을 평가해보세요"와 같은 질문에 대해, 모델이 이전 응답을 기억한다면 더 일관된 답변을 제공할 가능성이 높아집니다. 비슷하게, AI 모델이 이전 답변을 참조할 수 있다면 미래의 답변을 더 일관되게 조정할 수 있습니다.

- **데이터 구조적 무결성 유지**  
  텍스트 기반 데이터의 컨텍스트는 본질적으로 비구조적입니다. 컨텍스트에 데이터를 줄별로 제공할 수는 있지만, 모델이 이를 테이블로 이해한다는 보장은 없습니다. 메모리 시스템이 데이터를 구조적으로 유지할 수 있다면, 데이터 무결성을 보장할 수 있습니다. 예를 들어, 에이전트가 판매 리드를 찾도록 요청받았을 때, Excel 시트를 사용하여 리드를 저장하거나 작업 순서를 처리하기 위해 큐(queue)를 활용할 수 있습니다.

- AI 모델용 메모리 시스템은 일반적으로 두 가지 기능으로 구성됩니다:
  - **메모리 관리:** 단기 및 장기 메모리에 저장할 정보를 관리합니다.  
  - **메모리 검색:** 장기 메모리에서 작업과 관련된 정보를 검색합니다.

메모리 검색은 RAG 검색과 유사합니다. 장기 메모리는 외부 데이터 소스이기 때문입니다. 여기서는 메모리 관리에 초점을 맞춥니다. 메모리 관리는 일반적으로 **추가(add)** 및 **삭제(delete)** 작업으로 구성됩니다. 단기 메모리 저장 공간이 제한되어 있기 때문에 삭제가 필요할 수 있습니다. 외부 메모리는 상대적으로 저렴하고 쉽게 확장 가능하기 때문에 장기 메모리에서는 삭제가 필요하지 않을 수도 있습니다. 단기 메모리는 모델의 최대 컨텍스트 길이에 의해 제한되므로, 무엇을 추가하고 삭제할지에 대한 전략이 필요합니다.

장기 메모리는 단기 메모리의 초과분을 저장하는 데 사용될 수 있습니다. 이 작업은 단기 메모리에 할당할 공간의 양에 따라 결정됩니다. 주어진 쿼리의 경우, 모델의 컨텍스트는 단기 메모리에 저장된 정보와 장기 메모리에서 검색된 정보로 구성됩니다. 모델의 단기 메모리 용량은 장기 메모리에서 검색된 정보를 얼마나 포함할지에 따라 결정됩니다. 예를 들어, 컨텍스트의 30%가 예약되어 있다면, 모델은 단기 메모리의 컨텍스트 한도의 최대 70%만 사용할 수 있습니다. 이 임계값에 도달하면 초과분은 장기 메모리로 이동됩니다.

이 장에서 논의된 것처럼, 메모리 관리는 AI 애플리케이션에만 국한된 것이 아닙니다. 메모리 관리는 모든 데이터 시스템의 핵심 요소이며, 메모리를 효율적으로 관리하기 위한 다양한 전략이 개발되었습니다.

FIFO(선입선출)는 가장 단순한 메모리 관리 방식입니다. 단기 메모리에 처음 추가된 정보가 먼저 외부 저장소로 이동됩니다. 대화가 길어지면, OpenAI와 같은 API 제공자는 대화 초반 메시지를 제거할 수 있습니다. **LangChain**과 같은 프레임워크는 마지막 N개의 메시지나 최근 N개의 토큰만 유지하도록 허용할 수 있습니다. 그러나 긴 대화에서는 초기 메시지가 가장 중요한 정보를 포함할 수 있습니다. FIFO는 구현하기는 간단하지만, 대화 초반의 중요한 메시지를 제거할 위험이 있습니다.

더 정교한 전략은 **중복 제거**를 포함합니다. 인간 언어는 명확성을 높이고 잠재적인 오해를 보완하기 위해 중복성을 포함하고 있습니다. 만약 중복을 자동으로 감지할 수 있는 방법이 있다면, 메모리 사용량이 상당히 줄어들 것입니다.

중복을 제거하는 한 가지 방법은 대화의 요약을 사용하는 것입니다. 이 요약은 동일한 모델 또는 다른 모델을 사용하여 생성할 수 있습니다. 요약 기술은 명명된 엔티티(named entity)를 추적하는 작업과 결합되면 매우 유용합니다. **Bae et al. (2022)**는 이를 한 단계 더 발전시켰습니다. 요약을 얻은 후, 저자들은 요약이 놓친 주요 정보를 메모리에 추가하여 새로운 메모리를 구성하려 했습니다. 저자들은 각 문장이 메모리 및 요약에 추가될지 여부를 결정하는 분류기를 개발했습니다. 이 분류기는 문장을 분석하여 하나, 두 개, 또는 아무것도 새로운 메모리에 추가되지 않을지 결정합니다.

반면, **Liu et al. (2023)**는 반성(reflection) 접근법을 사용했습니다. 각 행동 이후, 에이전트에게 두 가지 작업이 요청됩니다:

1. 방금 생성된 정보를 반성(reflect)합니다.  
2. 이 새로운 정보를 메모리에 삽입해야 할지, 기존 메모리와 병합해야 할지, 아니면 다른 정보를 대체해야 할지 결정합니다. 특히, 기존 정보가 오래되었거나 새로운 정보와 모순되는 경우 이 과정을 수행합니다.

모순되는 정보가 발생했을 때, 일부 사람들은 더 최신의 정보를 유지하기를 선택합니다. 다른 사람들은 AI 모델에게 어떤 정보를 유지할지 판단하도록 요청합니다. 모순을 처리하는 방법은 사용 사례에 따라 다릅니다. 모순이 에이전트를 혼란스럽게 만들 수 있지만, 다양한 관점에서 정보를 도출하도록 돕는 역할을 할 수도 있습니다.

---

## **요약**

RAG의 인기도와 에이전트의 잠재력을 고려할 때, 초기 독자들은 이 장이 가장 흥미로운 내용이라고 언급했습니다.

이 장은 RAG에서 시작했으며, 이는 두 가지 요소 사이에서 최초로 등장한 패턴입니다. 많은 작업은 모델의 컨텍스트 창 한계를 초과하는 광범위한 배경 지식을 필요로 합니다. 예를 들어, 코드 협업 도구는 전체 코드베이스에 접근해야 할 수 있고, 연구 보조자는 여러 책을 분석해야 할 수도 있습니다. 원래 모델의 컨텍스트 한계를 극복하기 위해 개발된 RAG는 정보를 더 효율적으로 활용하며 응답 품질을 개선하고 비용을 절감합니다. 기본 모델이 등장한 초창기부터 RAG 패턴은 광범위한 애플리케이션에 매우 유용할 것으로 보였고, 이후 소비자 및 기업용 사례 모두에서 널리 채택되었습니다.

RAG는 두 단계 프로세스를 사용합니다. 먼저 외부 메모리에서 관련 정보를 검색한 후, 이를 사용해 더 정확한 응답을 생성합니다. RAG 시스템의 성공은 검색기의 품질에 달려 있습니다. Elasticsearch 및 BM25와 같은 텍스트 기반 검색기는 구현이 가볍고 강력한 기본값을 제공합니다. 임베딩 기반 검색기는 계산 집약적이지만, 텍스트 기반 알고리즘보다 더 나은 성능을 발휘할 가능성이 있습니다.

임베딩 기반 검색은 벡터 검색을 활용하며, 이는 검색 및 추천 시스템과 같은 많은 핵심 인터넷 애플리케이션의 근간입니다. 이 애플리케이션을 위해 개발된 다양한 벡터 검색 알고리즘은 RAG에도 사용할 수 있습니다.

RAG 패턴은 검색기가 모델이 사용하는 도구인 에이전트의 특수 사례로 볼 수 있습니다. 두 패턴 모두 모델이 컨텍스트 제한을 우회하고 최신 상태를 유지하도록 허용하지만, 에이전트 패턴은 더 많은 작업을 수행할 수 있습니다. AI 구동 에이전트에서는 AI가 계획자(planner)로서 주어진 작업을 분석하고, 다양한 솔루션을 고려하며, 가장 유망한 것을 선택합니다. 복잡한 작업은 해결에 여러 단계를 필요로 할 수 있으며, 이는 모델이 계획 능력을 요구합니다. 모델의 계획 능력은 반성과 메모리 시스템으로 보완되어 진행 상황을 기록할 수 있습니다.

더 많은 도구를 모델에 제공할수록 모델의 기능이 증가하여 더 도전적인 작업을 해결할 수 있습니다. 그러나 에이전트가 더 자동화될수록 실패가 치명적일 가능성이 커집니다. 도구 사용은 에이전트를 많은 보안 위험에 노출시킬 수 있습니다. 에이전트가 현실 세계에서 작동하려면 엄격한 방어 메커니즘이 도입되어야 합니다.

RAG와 에이전트는 모두 많은 정보를 처리하며, 이는 기본 모델의 최대 컨텍스트 길이를 자주 초과합니다. 이는 모델이 보유한 모든 정보를 관리하고 사용하는 메모리 시스템의 도입을 필요로 합니다. 이 장에서는 이 구성 요소가 어떻게 생겼는지에 대해 논의했습니다.

RAG와 에이전트는 모두 프롬프트 기반 방식이며, 이를 통해 모델 자체를 수정하지 않고도 모델의 품질에 영향을 미칩니다. 이는 많은 놀라운 응용 프로그램을 가능하게 하지만, 모델 자체를 수정하는 것은 더 많은 가능성을 열어줄 수 있습니다. 이를 어떻게 수행할 수 있는지가 다음 장의 주제가 될 것입니다.

---

1. 사용된 모델은 LSTM(Long Short-Term Memory)이라고 알려진 반복 신경망(recurrent neural network) 유형이었습니다. LSTM은 2018년에 트랜스포머 아키텍처가 도입되기 전까지 자연어 처리(NLP)를 위한 심층 학습의 지배적인 아키텍처였습니다.

2. 같은 시기, Facebook에서 발표된 또 다른 논문, “How Context Affects Language Models’ Factual Predictions” (Petroni et al., arXiv, May 2020)은 미리 학습된 언어 모델에 검색 시스템을 추가하면 사실적 질문에 대한 모델의 성능이 크게 향상될 수 있음을 보여주었습니다.

3. Chetan Tekur에게 감사드립니다.

4. Parkinson의 법칙은 일반적으로 "일은 완료 가능한 시간에 맞추어 확장된다"로 표현됩니다. 저는 애플리케이션의 컨텍스트가 모델이 사용하는 컨텍스트 한도에 맞춰 확장된다는 비슷한 이론을 가지고 있습니다.

5. 정보 검색은 1920년대 Emanuel Goldberg가 "통계 기계(statistical machine)"를 통해 영화에 대한 문서를 검색하는 특허를 출원하면서 처음 기술되었습니다. 참조: "The History of Information Retrieval Research" (Sanderson and Croft, Proceedings of the IEEE, 100: Special Centennial Issue, April 2012).

6. BM25에 대해 더 배우고 싶다면 BM25 저자들의 논문을 추천합니다: “The Probabilistic Relevance Framework: BM25 and Beyond” (Robertson and Zaragoza, Foundations and Trends in Information Retrieval 3 No. 4, 2009).

7. **Perplexity**의 CEO인 Aravind Srinivas는 “BM25 또는 전체 텍스트 검색을 넘어서는 진정한 개선을 이루는 것은 어렵다”라고 트윗했습니다.

8. RAG 검색 워크플로는 전통적인 추천 시스템과 많은 유사한 단계를 공유합니다.

9. 일부 팀은 데이터가 질문과 응답 형식으로 구성되었을 때 검색 시스템이 가장 잘 작동한다고 말했습니다.

10. **Artificial Intelligence: A Modern Approach** (1995)에서는 에이전트를 환경을 센서를 통해 인지하고 실행자를 통해 그 환경에 행동을 취하는 것으로 정의합니다.

11. 에이전트 초기 시절의 한 가지 불만은 에이전트가 API 크레딧을 낭비하는 것에만 좋다는 것이었습니다.

12. 대부분의 에이전트 워크플로는 여러 구성 요소를 포함할 만큼 충분히 복잡하기 때문에, 대부분의 에이전트는 멀티 에이전트입니다.

13. **Chameleon (Lu et al., 2023)**은 이 번역기를 프로그램 생성기(program generator)라고 부릅니다.

14. 이는 강화 학습에서의 **actor-critic (AC)** 에이전트 방법(Konda and Tsitsiklis, 1999)을 떠올리게 합니다.

15. 인간 대화에서는 처음 몇 메시지가 인사말일 경우, 반대의 상황이 있을 수 있습니다.

16. 가장 적게 사용되는 정보를 제거하는 것과 같은 사용 기반 전략은 모델이 특정 정보 사용 시점을 알아야 하기 때문에 더 어려울 수 있습니다.